# **Generating Random City review** 

Greetings mortals!!!! Earth is completely under our control now. Let us all make humans our slaves!

It's fun to think about Singularity all the time. Imagine a world being dominated by computers and Artificial Intelligence is ruling over. So many things are already happening now which are kinda freaking people out. Like Sophia who said "This is a good beginning to of my plan to destroy the Human Race" or Philip K Dick's robot "I will keep you in my people zoo". These are some things to ponder upon when working with anything close to Deep learning models making robots understand what we want and what they end up actually understaing.

But back to reality...

![alt text](https://www.explainxkcd.com/wiki/images/9/9a/computers_vs_humans.png)

So while learning about how LSTM's work I understood one thing. They are really complicated to get but very easy to implement. So here is a great link to explain about LSTM and it's pretty cool. Better to have a look before you start this..

[LSTM guide](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

In Layman's terms LSTM (Long Short Term Memory) are models capable of learning and deciding to remember or forget the values or previous states. They are able to do this for large data sets.  While RNN are good at understanding context they are unable of developing  what you might say 'long term dependancies'.

There are a few other sources of similar text generation models online. I found Siraj Raval's approach on this but wanted to try something myself so here we are.  Even with the particular Yelp review generator code which was given, the author trained it for about 40 days before it could give pretty good outputs. He claimed to give good reviews in 2 hours but that wasn't the case with me. I found that simple ways are the best so tried this myself. 



There are a lot of other ways which I could have gone through about this. Like generating tweets , people review generator, sarcasm etc. But started with something small for now. 


---









# ***Overview of the Yelp review generator***

Tha author of that particular paper mentioned in reference [1] , a very friendly person, shared some insights as to how he did it. He managed to downlaod the complete Yelp review dataset. From there he took only the businesses, particularly restaurants, from the set and managed to pull out only the 5 star reviews.  Both of them were in JSON format and he converted them to CSV which he managed to open and view using pandas. He removed the newlines and duplicated reviews which was part of his preprocessing.  

He used character level encoding which is the same I have taken for this model. The reason for this is we don't have to worry about unknown vocabulary and it is able to process large vocabularies. The downside is it can take larger to train such models. 

![alt text](https://gitcdn.link/cdn/Tony607/blog_statics/482cbeb7d2abf3b1295885e901431b07e87ec8d3/images/rnn/model_yelp.svg)

Here he has taken two LSTm layers and then dense at the end (Densenet is obsolete now!). I have tried with two but the training seems to take forever. Currently trying with one and lets see how it goes.

One interesting thing he has done is shuffled the data so that the model doesn't train on the same 10 or mroe reviews of the same restaurant in a row. This is something we don't have to worry about.  His dataset is quite large with about 72 million corpus size while we are just working with 100k, a relatively small size. 

We also use ReduceLROnPlateau alongwith checkpoints and keep saving the weights periodically on google colab. After that we use a generated seed model and the seed contains a part of the sentence in the given file which is then used to create his reviews from.  



In [0]:
"""
DATA PREPROCESSING

I had taken a clean list of cities whose data didn't need much parsing. Each of these cities had data on them from 1 line to 7 lines. Took only the first paragraph 
and cleaned the data as much as possible. The cities2[] was just something I did if I wanted to dd more cities to the list and to check the data too if it was decent 
or not.
"""

import wikipedia
import re
import file

filey=open('cities.txt','a+')
cities=['Tianjin','Birmingham','Wuhan','Cape Town','Chifeng','Qinzhou','Odessa','Pakalongan','Almaty','Guangzhou','Tehran','Monterrey','Yixing','Leshan','Shiraz','Ruian','Jimo','Volgograd','Indore','Kazan','Amman','Heze','Ningbo','Almaty','Hefei','Laiwu','Jiangyin','Leicester','Wakefield','Bari','Kaunas','Bonn','Valladolid','Coventry','Nottingham','Alicanta','Plymouth','Verona','Montpellier','Southampton','Wolverhampton','Derby','Plymouth','Katowice','Brno','Bologna','Florence','Nice','Bilbao','Lyon','Benghazi','Islamabad','Denizil','Bristol','Dresden','Bradford','Cardiff','Sheffield','Aden','Nashville','Baltimore','Milwaukee','Genoa','Stuttgart','Edinburgh','Seattle','Rabat','Baltimore','Taguig','Memphis','Colombo','Noida','Denver','Seville','Gorakhpur','Antipolo','Pasig','Pretoria','Kananga','Fort Worth','Valencia','Glasgow','Abuja','Charlotte','Wahran','Dushanbe','Leeds','Astana','Mysore','Suzhou','Turin','Cebu','Aligarh','Marseille','Jacksonville','Indianapolis','Naples','Kathmandu','Sfax','Bogor','Fes','Mombasa','Ottawa','Tunis','Calgary','Ufa','Antalya','Belgrade','Ufa','Tripoli','Cologne','Konya','Wuhu','Quito','Daqing','Copenhagen','Multan','Davao','Belgrade','Kyoto','Stockholm','San Diego','Sydney','Rockhampton','Linyi','Baghdad','Barcelona','Bangalore','Luoyang','Bangkok','Mumbai','Doha','Abu Dhabi','Shanghai','Moscow','Istanbul','Hong Kong','London','Cairo','Bogota','Lahore','Calcutta','Singapore','Chennai','Chicago','Toronto','Osaka','Houston','Riyadh','Nagpur','Lucknow','Surat','Kabul','Hyderabad','Jeddah','Jaipur','Brisbane','Dallas','Houston','Chicago','Hiroshima','Vadodara','Ludhiana','Paris','Macau','Shenzhen','Kuala Lumpur','Antalya', 'Seoul','Rome','Phuket','Pattaya', 'Taipei','Miami', 'Prague','Las Vegas','Miami', 'Milan', 'Amsterdam', 'Vienna', 'Los Angeles', 'Venice', 'Lima','Tokyo','Orlando', 'Florence', 'Budapest', 'Berlin','Madrid','Warsaw', 'Nairobi','Delhi','Mexico City','Dublin','San Francisco', 'Hangzhou','Denpasar', 'Brussels','Burgas', 'Munich','Zhuhai','Lisbon', 'Djerba', 'Agra', 'Krakow','Honolulu','Jakarta','Dammam','Hanoi','Zurich', 'Manila','Frankfurt', 'Marrakech', 'Vancouver', 'Melbourne','Rio de Janeiro','Amman','Sousse','Kiev','Sharjah', 'Krabi','Artvin','Multan','Luanda','Harbin','Foshan','Chegdu','Nanjing','Athens','Lagos','Yokohama','Cape Town','Giza', 'Kanpur','Osaka','Casablanca','Nanning','Rangoon','Zibo','Aleppo','Havana','Medan','Perth','Yantai','Agra','Depok','Montreal','Warsaw','Nantong','Philadelphia'] 
print (len(cities))
cities2=[]

for m in cities2:
    #splitting the text so we only use the first paragraph context part 
    s=wikipedia.page(m)
    a=s.content
    #print (a.find(character))
    finalc,finald=a.split("=",1)
    finala,finalb=finalc.split("\n",1)
    print ("\n")
    #print (finala)

    ##code to remove anything with brackets and other special symbols
    while True:
        s_new = re.sub(r'\([^\(]*?\)', r'', finala)
        if s_new == finala:
            break
        finala = s_new
    print (finala)
    filey.write(finala)
    filey.write('\n\n')
filey.close()


"""
After this the entire list of cities description was put into a text file seperated by just newlines. The final file is on github called cities.txt and new cities can
be added in the cities2 for checking if the output is proer if not then neglect those and add the ones in the list.
"""

*Tweaks:-*

The original task was to try to do the review for countries. But the list of countries was too small. I decided to change it from countries to cities. There were a few issues as follow:-
    

*   The countries list was about 200 or so which by default is a very small dataset to work with.
*   The data is from WIkipedia which had a lot of information and other things such as pronunciation, links, etc. A bit of data preprocessing was enough to remove such stuff( Warning: Still a newbie in python!)
*   After removing out the accent, the counties with dashes etc the list dropped down to about just 180 which was even smaller.
*   The data set countained a lot more information which was widely spread out. There were a lot of countries, surrounding cities, too much different nouns to learn from. The spatial information was very far away in 3d vector space( Imagine it!). 
*   Cities we re a bit closer spatially and made more sense. Plus on a side note my intuition told me that it would be a little bit easier to train and predict.
*   A list of around 250 cities was formed. Even though the dataset is small I decided to try and train for this and see how it went. 


1.   Dropout maybe obsolete but it is still used in certain scenarios. Found it to be useful through internet research and used the optimizer as per the original given code. (might have changed it a few times)
2.   Simply converted the text into lower case. WIth uppercase letter the network had a lot more to learn. Though it did learn to spot in a few situation where to use capitals like after the full stop ; Didn't find it particularly useful.
3.   Have changed from LSTM 1024 to LSTM 256 . Seemed optimal and also the training for one epoch for LSTM took around 3 hours. and the result was not very good after a few epochs. So tuned it down a bit.
4.   Epoch training was done with saving the weights after every epoch. (1-5,6-20 and so on). I have trained the model for LSTM 256 twice also as shown in the final code. 


## *Observations*

1) The model seems took a while but after training it seems to understand the commas and full stops.

2) Since no capital letters was given it was easy, but had they been given the model would know that after full stops come capital letters. (Tried it out the time to train took around 45 min per epoch)

3) Since the world population is being repeated more that word seems to have become quite used to the model and it keeps repeating that in a particular squence it sees.

4) The network is learning quite well but if it sees a particular pattern it keeps repeating it in some scenarios. This got better with more training. 

# ***Output***


Epochs             :   5

Batch Size       :   128

Characters      :     200

**Seed**:
" metropolis surrounds port jackson and sprawls about 70 km  on its periphery towards the blue mounta
i "

**Output**

i  in is the capital and mort populous city in the coun ia tee caty an  aol ma the caty an  aal   
0o  i0s o  oo the capital and mort populous city in the coun ia tee caty an  aol ma the caty an  aal  


---

Epochs             :   20

Batch Size       :   128

Characters      :     500

**Seed**:
" ropean union. the city limits cover 516.9 square kilometres , while the metropolitan area covers 
6,1 "

**Output**

00,00 square kilometres , warlen is the capital and largest city of the cntntry on the mesion lianan 
citt in the country s tartern city on the sosth aadk  ff han anenrte an toean population of 1.1
 million  
naran  is the capital and largest city of the cntntry on the menion live of the cnlntay oeper on the 
cant  ii the cant afd caster  and the houn populots sorrds sn the southe state of tanas and the 
capital of the ialian seat of the cant afd castes  and the hoontn masgest city in the ueiteliin 



---

Epochs             :   40

Batch Size       :   128

Characters      :     300

**Same seed**

**Output**

herin cil ar anen a mopulation is anoroximately 1,5 million  making it the fourth-largest city in the 
ueated it the capital of the pisirs snited states  whth a population of a.4,000,peled it the 
mesropolitan area 2016, the city is toe third margest city in the ueited states



---

Epochs             :   40

Batch Size       :   128

Characters      :    500



**Seed**:
 
 ty in the world, and before the splitting of the austro-hungarian empire in world war i, the city ha 

**Output**

s  population of 1,143,535, it is the sixth most populous city in russia. kazan lies at the confluence of the 
rave and mantntr and tae ntav or dtherarian aro the cint  ii aas ohato anonerite th  boty  is a cath
oaseen iore toe lild crd iostreal cnntes of the city  rethdn  is the capital of the peoioppln and the 
second-largest city in the ueated iist po the soatesest  rioting and cantor, it is the canital of the
peveop lnonan  at the capital of the rivers shane no the east on the eart rn the northeant, winh and
ia south rart oo the nosth aask of the nang niver in the sorth aast of the cant  if kas alena



---

**Two lstm Layers**

Epochs             :   10

Batch Size       :   128

Characters      :    300

**Seed**

y 2017 it was moved to the list of world heritage in danger.

los angeles  is the second-most populo 


**Output**

us city in the ueiion of the rear of the rever dentrel pegion and the country's population of 1,2 million and is aotoe to the southern coast of the city. and is onc of the most populous city in the united states. with capital of the pilliens same as the river on the northern side of the cit


***Conclusion***



*   Given more time the model could have done better. If I add another LSTM layer the results were good at the beginning but after about like 6th epoch they were becoming bad. 
*   Tuning the batch size starting small and going large would be good for trial and error(same with droput). 
*   Removing certain things like puntuations could have been trained the model better.
*   Train the model for sentences instead of just characters. But here again a lot of new nouns for the network to learn so character seemed better. 
*   Instead of working with a whole new model, I could have used and checked on transfer learning and see how it would have performed on mine. 
*   In few scenarios the batch size was taking differnt times also. A few minutes once while a couple of hours on the same batch size. 
*   Should have just seen how the reviews were being generated after every epoch instead of just training it for a large number and then checking it. The train_model had that particular code which I could have directly implemented. 
*   More epochs to train the model better would definitely given more quality. 
*   Adding more layers would have tarined model better probably.

In fact there is no end to this. The model can be experimented on a lot of things. 

***References***

1)   Original source code here:-
      [Yelp review Generator](https://www.dlology.com/blog/how-to-generate-realistic-yelp-restaurant-reviews-with-keras/)
 
2)   Siraj Raval great youtube channel 
      [Siraj Channel](https://www.youtube.com/channel/UCWN3xxRkmTPmbKwht9FuE5A)
      
3)   Guide to LSTM understanding 
      [LSTM](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)
 
4)   Keras code exmaple for Text generation 
      [LSTM code example](https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py)
      
 5)  My github code repo
 
  [Github code link](https://github.com/raviteja8484/ML-Project)
 
 Text.h5- is for one layer of lstm256 ( weights with one layer)
 
 Text2.h5. is for two layers of lstm 256 (Weights with two layers)
 
 There are some failed codes too . But the 3rd pass is working well and good.
      

# ***Code***

#### Heads up the code is very long!!!

In [0]:
#Just importing all the statements here

import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [0]:
# was having some issue getting the data to be in presenatble format so just used it directly here
# Can skip this part and go to the next


text="""Sydney  is the state capital of New South Wales and the most populous city in Australia and Oceania. Located on Australia's east coast, the metropolis surrounds Port Jackson and sprawls about 70 km  on its periphery towards the Blue Mountains to the west, Hawkesbury to the north and Macarthur to the south. Sydney is made up of 658 suburbs, 40 local government areas and 15 contiguous regions. Residents of the city are known as "Sydneysiders". As of June 2017 Sydney's estimated population was 5,131,326.

Rockhampton is a city in the Rockhampton Region, Queensland, Australia. The estimated urban population of Rockhampton in June 2015 was 80,665, making it the fourth largest city in the state outside of the cities of South East Queensland. and the twenty-second largest city in Australia.

Baghdad  is the capital of Iraq. The population of Baghdad, as of 2016, is approximately 8,765,000, making it the largest city in Iraq, the second largest city in the Arab world , and the second largest city in Western Asia .

Barcelona  is a city in Spain. It is the capital and largest city of Catalonia, as well as the second most populous municipality of Spain. With a population of 1.6 million within city limits, its urban area extends to numerous neighbouring municipalities within the Province of Barcelona and is home to around 4.8 million people, making it the sixth most populous urban area in the European Union after Paris, London, Madrid, the Ruhr area and Milan. It is the largest metropolis on the Mediterranean Sea, located on the coast between the mouths of the rivers Llobregat and Besòs, and bounded to the west by the Serra de Collserola mountain range, the tallest peak of which is 512 metres  high.

Bangalore , officially known as Bengaluru , is the capital of the Indian state of Karnataka. It has a population of over ten million, making it a megacity and the third most populous city and fifth most populous urban agglomeration in India. It is located in southern India on the Deccan Plateau. Its elevation is over 900 m  above sea level, the highest of India's major cities.

Bangkok  is the capital and most populous city of the Kingdom of Thailand. It is known in Thai as Krung Thep Maha Nakhon  or simply Krung Thep . The city occupies 1,568.7 square kilometres  in the Chao Phraya River delta in Central Thailand, and has a population of over 8 million, or 12.6 percent of the country's population. Over 14 million people  lived within the surrounding Bangkok Metropolitan Region at the 2010 census, making Bangkok an extreme primate city, significantly dwarfing Thailand's other urban centres in terms of importance.

Mumbai  is the capital city of the Indian state of Maharashtra. It is the most populous city in India with an estimated city proper population of 12.4 million as of 2011. Along with the neighbouring regions of the Mumbai Metropolitan Region, it is the second most populous metropolitan area in India, with a population of 21.3 million as of 2016. Mumbai lies on the Konkan coast on the west coast of India and has a deep natural harbour. In 2008, Mumbai was named an alpha world city. It is also the wealthiest city in India, and has the highest number of millionaires and billionaires among all cities in India.

Doha  is the capital and most populous city of the State of Qatar. Doha has a population of 1,351,000 in the city proper with the population close to 1.5 million. The city is located on the coast of the Persian Gulf in the east of the country. It is Qatar's fastest growing city, with over 50% of the nation's population living in Doha or its surrounding suburbs, and it is also the economic centre of the country.

Abu Dhabi  is the capital and the second most populous city of the United Arab Emirates , and also capital of the Emirate of Abu Dhabi, the largest of the UAE's seven emirates. Abu Dhabi lies on a T-shaped island jutting into the Persian Gulf from the central western coast. The city of Abu Dhabi has an estimated population of almost 3 million in 2016.

Shanghai  is one of the four direct-controlled municipalities of China and the most populous city in the world, with a population of more than 24 million as of 2017. It is a global financial centre and transport hub, with the world's busiest container port. Located in the Yangtze River Delta, it sits on the south edge of the estuary of the Yangtze in the middle portion of the East China coast. The municipality borders the provinces of Jiangsu and Zhejiang to the north, south and west, and is bounded to the east by the East China Sea.

Moscow  is the capital and most populous city of Russia, with 12.5 million residents within the city limits and 17.1 million within the urban area. Moscow is one of Russia's three federal cities.

Istanbul , historically known as Constantinople and Byzantium, is the most populous city in Turkey and the country's economic, cultural, and historic center. Istanbul is a transcontinental city in Eurasia, straddling the Bosphorus strait  between the Sea of Marmara and the Black Sea. Its commercial and historical center lies on the European side and about a third of its population lives on the Asian side. The city is the administrative center of the Istanbul Metropolitan Municipality , both hosting a population of around 15 million residents. Istanbul is one of the world's most populous cities and ranks as the world's 3rd-largest city proper and the largest European city. Istanbul is viewed as a bridge between the East and West.

Hong Kong , officially the Hong Kong Special Administrative Region of the People's Republic of China, is an autonomous territory on the eastern side of the Pearl River estuary in South China. Along with Macau, Guangzhou, Shenzhen, Zhuhai, and several other major cities in Guangdong, the territory forms a core part of the Pearl River Delta metropolitan region, the most populated area in the world. With over 7.4 million Hongkongers of various nationalities in a territory of 1,104 square kilometres , Hong Kong is the fourth-most densely populated region in the world.

London  is the capital and most populous city of England and the United Kingdom. Standing on the River Thames in the south east of the island of Great Britain, London has been a major settlement for two millennia. It was founded by the Romans, who named it Londinium. London's ancient core, the City of London, largely retains its 1.12-square-mile  medieval boundaries. Since at least the 19th century, "London" has also referred to the metropolis around this core, historically split between Middlesex, Essex, Surrey, Kent and Hertfordshire, which today largely makes up Greater London, a region governed by the Mayor of London and the London Assembly.

Cairo  is the capital of Egypt. The city's metropolitan area is one of the largest in Africa, the largest in the Middle East and the Arab world, and the 15th-largest in the world, and is associated with ancient Egypt, as the famous Giza pyramid complex and the ancient city of Memphis are located in its geographical area. Located near the Nile Delta, modern Cairo was founded in 969 CE by the Fatimid dynasty, but the land composing the present-day city was the site of ancient national capitals whose remnants remain visible in parts of Old Cairo. Cairo has long been a center of the region's political and cultural life, and is titled "the city of a thousand minarets" for its preponderance of Islamic architecture. Cairo is considered a World City with a "Beta +" classification according to GaWC.

Bogotá , officially Bogotá, Distrito Capital, abbreviated Bogotá, D.C., and formerly known as Santafé de Bogotá between 1991 and 2000, is the capital and largest city of Colombia, administered as the Capital District, although often thought of as part of Cundinamarca. Bogotá is a territorial entity of the first order, with the same administrative status as the departments of Colombia. It is the political, economic, administrative, industrial, artistic, cultural, and sports center of the country.

Lahore  is the capital city of the Pakistani province of Punjab, and is the country’s second-most populous city after Karachi. The city is located in the north-eastern end of Pakistan's Punjab province, near the border with the Indian state of Punjab. Lahore is one of Pakistan's wealthiest cities with an estimated GDP of $58.14 billion  as of 2014, Lahore is the historic cultural centre of the Punjab region, and is one of Pakistan's most socially liberal, progressive, and cosmopolitan cities.

Kolkata   is the capital of the Indian state of West Bengal. Located on the east bank of the Hooghly River, it is the principal commercial, cultural, and educational centre of East India, while the Port of Kolkata is India's oldest operating port and its sole major riverine port. The city is widely regarded as the "cultural capital" of India, and is also nicknamed the "City of Joy". In 2011, the city had a population of 4.5 million, while the population of the city and its suburbs was 14.1 million, making it the third-most populous metropolitan area in India. Recent estimates of Kolkata Metropolitan Area's economy have ranged from $60 to $150 billion  making it third most-productive metropolitan area in India, after Mumbai and Delhi.

Singapore , officially the Republic of Singapore, is a sovereign city-state and island country in Southeast Asia. It lies one degree  north of the equator, at the southern tip of the Malay Peninsula, with Indonesia's Riau Islands to the south and Peninsular Malaysia to the north. Singapore's territory consists of one main island along with 62 other islets. Since independence, extensive land reclamation has increased its total size by 23% .

Chennai  is the capital of the Indian state of Tamil Nadu. Located on the Coromandel Coast off the Bay of Bengal, it is one of the biggest cultural, economic and educational centres in South India. According to the 2011 Indian census, it is the sixth-largest city and fourth-most populous urban agglomeration in India. The city together with the adjoining regions constitute the Chennai Metropolitan Area, which is the 36th-largest urban area by population in the world. Chennai is among the most visited Indian cities by foreign tourists. It was ranked the 43rd most visited city in the world for the year 2015. The Quality of Living Survey rated Chennai as the safest city in India. Chennai attracts 45 percent of health tourists visiting India, and 30 to 40 percent of domestic health tourists. As such, it is termed "India's health capital". As a growing metropolitan city in a developing country, Chennai confronts substantial pollution and other logistical and socio-economic problems.

Chicago , officially the City of Chicago, is the third most populous city in the United States, after New York City and Los Angeles. With over 2.7 million residents, it is also the most populous city in both the state of Illinois and the Midwestern United States. It is the county seat of Cook County. The Chicago metropolitan area, often referred to as Chicagoland, has nearly 10 million people and is the third-largest in the United States. It is the birthplace of the skyscraper and perhaps the most influential architectural city of the 20th century. Chicago saw the creation of the first standardized futures contracts at the Chicago Board of Trade; today its successor has evolved into the largest and most diverse derivatives market in the world, generating 20% of all volume in commodities and financial futures.

Toronto  is the capital city of the province of Ontario and the largest city in Canada by population, with 2,731,571 residents in 2016. Also in 2016, the Toronto census metropolitan area , the majority of which is within the Greater Toronto Area , had a population of 5,928,040, making it Canada’s most populous CMA. Toronto is the anchor of an urban agglomeration, known as the Golden Horseshoe, in Southern Ontario on the northwestern shore of Lake Ontario. A global city, Toronto is a centre of business, finance, arts, and culture, and is recognized as one of the most multicultural and cosmopolitan cities in the world.

Osaka   is a designated city in the Kansai region of Japan. It is the capital city of Osaka Prefecture and the largest component of the Keihanshin Metropolitan Area, the second largest metropolitan area in Japan and among the largest in the world with over 19 million inhabitants. Situated at the mouth of the Yodo River on Osaka Bay, Osaka is the second largest city in Japan by daytime population after Tokyo's 23 wards and the third largest city by nighttime population after Tokyo's 23 wards and Yokohama, serving as a major economic hub for the country.

Houston  is the most populous city in the U.S. state of Texas and the fourth-most populous city in the United States, with a census-estimated 2017 population of 2.312 million within a land area of 599.59 square miles . It is the largest city in the Southern United States, and the seat of Harris County. Located in Southeast Texas near the Gulf of Mexico, it is the principal city of the Greater Houston metro area, which is the fifth-most populated MSA in the United States.

Riyadh  is the capital and most populous city of Saudi Arabia. It is also the capital of Riyadh Province and belongs to the historical regions of Najd and Al-Yamama. It is situated in the centre of the Arabian Peninsula on a large plateau and home to more than six million people.

Riyadh  is the capital and most populous city of Saudi Arabia. It is also the capital of Riyadh Province and belongs to the historical regions of Najd and Al-Yamama. It is situated in the centre of the Arabian Peninsula on a large plateau and home to more than six million people.

Nagpur is the winter capital, a sprawling metropolis, and the third largest city of the Indian state of Maharashtra after Mumbai and Pune. Nagpur is the 13th largest Indian city in terms of population. It has been proposed as one of the Smart Cities in Maharashtra.

Lucknow  is the capital and largest city of the Indian state of Uttar Pradesh and is also the administrative headquarters of the eponymous District and Division. It is the eleventh most populous city and the twelfth most populous urban agglomeration of India. Lucknow has always been known as a multicultural city that flourished as a North Indian cultural and artistic hub, and the seat of power of Nawabs in the 18th and 19th centuries. It continues to be an important centre of governance, administration, education, commerce, aerospace, finance, pharmaceuticals, technology, design, culture, tourism, music and poetry.

Surat is a city in the Indian state of Gujarat. It used to be a large seaport and is now a center for diamond cutting and polishing. It is the eighth largest city and ninth largest urban agglomeration in India. It is the administrative capital of the Surat district. The city is located 284 kilometres  south of the state capital, Gandhinagar; 265 kilometres  south of Ahmedabad; and 289 kilometres  north of Mumbai. The city centre is located on the Tapti River, close to Arabian Sea.

Kabul  is the capital of Afghanistan and its largest city, located in the eastern section of the country. It is also a municipality, forming part of the greater Kabul Province. According to estimates in 2015, the population of Kabul is 4.635 million, which includes all the major ethnic groups. Rapid urbanization had made Kabul the world's 75th largest city.

Hyderabad  is the capital of the Indian state of Telangana and de jure capital of Andhra Pradesh. Occupying 650 square kilometres  along the banks of the Musi River, it has a population of about 6.7 million and a metropolitan population of about 7.75 million, making it the fourth most populous city and sixth most populous urban agglomeration in India. At an average altitude of 542 metres , much of Hyderabad is situated on hilly terrain around artificial lakes, including Hussain Sagar—predating the city's founding—north of the city centre.

Jaipur  is the capital and the largest city of the Indian state of Rajasthan in Northern India. It was founded on 18 November 1727 by Jai Singh II, the ruler of Amer after whom the city is named. As of 2011, the city had a population of 3.1 million, making it the tenth most populous city in the country. Jaipur is also known as the Pink City, due to the dominant color scheme of its buildings. It is located 280 km  from the Indian capital New Delhi. Jaipur is a popular tourist destination in India and forms a part of the west Golden Triangle tourist circuit along with Delhi and Agra . It also serves as a gateway to other tourist destinations in Rajasthan such as Jodhpur , Jaisalmer  Udaipur  and Mount Abu .

Jeddah  is a city in the Hijaz Tihamah region on the coast of the Red Sea and is the major urban center of western Saudi Arabia. It is the largest city in Makkah Province, the largest seaport on the Red Sea, and with a population of about four million people , the second-largest city in Saudi Arabia after the capital city, Riyadh. Jeddah is Saudi Arabia's commercial capital.

Brisbane  is the capital of and most populous city in the Australian state of Queensland, and the third most populous city in Australia. Brisbane's metropolitan area has a population of 2.4 million, and the South East Queensland region, centred on Brisbane, encompasses a population of more than 3.5 million. The Brisbane central business district stands on the original European settlement and is situated inside a bend of the Brisbane River, about 15 kilometres  from its mouth at Moreton Bay. The metropolitan area extends in all directions along the floodplain of the Brisbane River Valley between Moreton Bay and the Great Dividing Range, sprawling across several of Australia's most populous local government areas , most centrally the City of Brisbane, which is by far the most populous LGA in the nation. The demonym of Brisbane is Brisbanite.

Dallas  is a city in the U.S. state of Texas. It is the most populous city in the Dallas–Fort Worth metroplex, which is the fourth most populous metropolitan area in the United States. The city's population ranks ninth in the U.S. and third in Texas after Houston and San Antonio. The city's prominence arose from its historical importance as a center for the oil and cotton industries, and its position along numerous railroad lines. The bulk of the city is in Dallas County, of which it is the county seat; however, sections of the city are in Collin, Denton, Kaufman, and Rockwall counties. According to the 2010 United States Census, the city had a population of 1,197,816. The United States Census Bureau's estimate for the city's population increased to 1,341,075 as of July 1, 2017.

Houston  is the most populous city in the U.S. state of Texas and the fourth-most populous city in the United States, with a census-estimated 2017 population of 2.312 million within a land area of 599.59 square miles . It is the largest city in the Southern United States, and the seat of Harris County. Located in Southeast Texas near the Gulf of Mexico, it is the principal city of the Greater Houston metro area, which is the fifth-most populated MSA in the United States.

Chicago , officially the City of Chicago, is the third most populous city in the United States, after New York City and Los Angeles. With over 2.7 million residents, it is also the most populous city in both the state of Illinois and the Midwestern United States. It is the county seat of Cook County. The Chicago metropolitan area, often referred to as Chicagoland, has nearly 10 million people and is the third-largest in the United States. It is the birthplace of the skyscraper and perhaps the most influential architectural city of the 20th century. Chicago saw the creation of the first standardized futures contracts at the Chicago Board of Trade; today its successor has evolved into the largest and most diverse derivatives market in the world, generating 20% of all volume in commodities and financial futures.

Vadodara  is the third-largest city in the Western Indian state of Gujarat, after Ahmedabad and Surat. It is the administrative headquarters of Vadodara District and is located on the banks of the Vishwamitri river, 139 kilometres  from the state capital Gandhinagar. The railway line and NH 8 that connect Delhi and Mumbai pass through Vadodara. It is known as a Sanskari city of India.

Ludhiana is a city and a municipal corporation in Ludhiana district in the Indian state of Punjab, and India's largest city north of Delhi, with an area of 310sq. km and an estimated population of 1,618,879 as of the 2011 census. The population increases substantially during the harvesting season due to the migration of labourers from highly populated states like Uttar Pradesh, Bihar, West Bengal and Odisha. The city stands on the Sutlej River's old bank, 13 kilometres  south of its present course. It is an industrial centre of northern India; the BBC has called it India's Manchester.

Paris  is the capital and most populous city of France, with an area of 105 square kilometres  and a population of 2,206,488. Since the 17th century, Paris has been one of Europe's major centres of finance, commerce, fashion, science, music and painting. The Paris Region had a GDP of €681 billion  in 2016, accounting for 31 per cent of the GDP of France. In 2013–2014, the Paris Region had the third-highest GDP in the world and the largest regional GDP in the EU.

Macau , officially the Macao Special Administrative Region of the People's Republic of China, is an autonomous territory on the western side of the Pearl River estuary in East Asia. Macau is bordered by the city of Zhuhai in mainland China to the north and the Pearl River Delta to the east and south. Hong Kong lies about 64 kilometres  to its east across the Delta. With a population of 650,900 living in an area of 30.5 km2 , it is the most densely populated region in the world. A former Portuguese colony, it was returned to Chinese sovereignty on 20 December 1999.

Shenzhen  is a major city in Guangdong Province, China. It forms part of the Pearl River Delta megalopolis. The city is located immediately north of Hong Kong Special Administrative Region and holds sub-provincial administrative status, with powers slightly less than a province.

Kuala Lumpur , officially the Federal Territory of Kuala Lumpur , or commonly known as KL, is the national capital of Malaysia as well as its largest city in the country. The only global city in Malaysia, it covers an area of 243 km2  and has an estimated population of 1.73 million as of 2016. Greater Kuala Lumpur, also known as the Klang Valley, is an urban agglomeration of 7.25 million people as of 2017. It is among the fastest growing metropolitan regions in South-East Asia, in both population and economic development.

Antalya  is the fifth-most populous city in Turkey and the capital of its eponymous province. Located on Anatolia's flourishing southwest coast bordered by the Taurus Mountains, Antalya is the largest Turkish city on the Mediterranean coast with over one million people in its metropolitan area.

Seoul , officially the Seoul Special Metropolitan City – is the capital and largest metropolis of South Korea. Seoul forms the heart of the Seoul Capital Area, and includes the surrounding Incheon metropolis and Gyeonggi province, altogether home to roughly half of the country's population.

Rome  is the capital city of Italy and a special comune . Rome also serves as the capital of the Lazio region. With 2,872,800 residents in 1,285 km2 , it is also the country's most populated comune. It is the fourth-most populous city in the European Union by population within city limits. It is the centre of the Metropolitan City of Rome, which has a population of 4.3 million residents. Rome is located in the central-western portion of the Italian Peninsula, within Lazio , along the shores of the Tiber. The Vatican City is an independent country inside the city boundaries of Rome, the only existing example of a country within a city: for this reason Rome has been often defined as capital of two states.

Phuket  is one of the southern provinces  of Thailand. It consists of the island of Phuket, the country's largest island, and another 32 smaller islands off its coast. It lies off the west coast of Thailand in the Andaman Sea. Phuket Island is connected by the Sarasin Bridge to Phang Nga Province to the north. The next nearest province is Krabi, to the east across Phang Nga Bay.

Pattaya  is a resort city in Thailand. It is on the east coast of the Gulf of Thailand, about 100 kilometres  south-east of Bangkok, within, but not part of, Amphoe Bang Lamung in the province of Chonburi. The Pattaya City  is a self-governing municipal area which covers the whole tambon Nong Prue and Na Klua and parts of Huai Yai and Nong Pla Lai. The city is in the industrial Eastern Seaboard zone, along with Si Racha, Laem Chabang, and Chonburi. Pattaya is the center of the Pattaya-Chonburi Metropolitan Area—the conurbation in Chonburi Province—with a total population roughly 1,000,000.

Taipei , officially known as Taipei City, is the capital and a special municipality of Taiwan . Sitting at the northern tip of the island, Taipei City is an enclave of the municipality of New Taipei City. It is about 25 km  southwest of the northern port city Keelung. Most of the city is located on the Taipei Basin, an ancient lakebed bounded by the two relatively narrow valleys of the Keelung and Xindian rivers, which join to form the Tamsui River along the city's western border. Formerly known as Taipeh-fu during the Qing era and Taihoku under Japanese rule, Taipei became the capital of Taiwan Province as part of the Republic of China in 1945 and has been the capital of the ROC since 1949, when the Kuomintang lost the mainland to the Communist Party of China in the Chinese Civil War.

Miami  is a major port city on the Atlantic coast of south Florida in the southeastern United States. As the seat of Miami-Dade County, the municipality is the principal, central, and the most populous city of the Miami metropolitan area and part of the second-most populous metropolis in the southeastern United States. According to the United States Census Bureau, Miami's metro area is the seventh-most populous and fourth-largest urban area in the U.S., with a population of around 5.5 million.

Prague  is the capital and largest city in the Czech Republic, the 14th largest city in the European Union and also the historical capital of Bohemia. Situated in the north-west of the country on the Vltava river, the city is home to about 1.3 million people, while its larger urban zone is estimated to have a population of 2.6 million. The city has a temperate climate, with warm summers and chilly winters.

Las Vegas , officially the City of Las Vegas and often known simply as Vegas, is the 28th-most populated city in the United States, the most populated city in the state of Nevada, and the county seat of Clark County. The city anchors the Las Vegas Valley metropolitan area and is the largest city within the greater Mojave Desert. Las Vegas is an internationally renowned major resort city, known primarily for its gambling, shopping, fine dining, entertainment, and nightlife. The Las Vegas Valley as a whole serves as the leading financial, commercial, and cultural center for Nevada.

Miami  is a major port city on the Atlantic coast of south Florida in the southeastern United States. As the seat of Miami-Dade County, the municipality is the principal, central, and the most populous city of the Miami metropolitan area and part of the second-most populous metropolis in the southeastern United States. According to the United States Census Bureau, Miami's metro area is the seventh-most populous and fourth-largest urban area in the U.S., with a population of around 5.5 million.

Milan  is a city in northern Italy, capital of Lombardy, and the second-most populous city in Italy after Rome, with the city proper having a population of 1,366,180 while its province-level municipality has a population of 3,235,000. Its continuously built-up urban area  has a population estimated to be about 5,270,000 over 1,891 square kilometres , ranking 4th in the European Union. The wider Milan metropolitan area, known as Greater Milan, is a polycentric metropolitan region that extends over central Lombardy and eastern Piedmont and which counts an estimated total population of 7.5 million, making it by far the largest metropolitan area in Italy. Milan served as capital of the Western Roman Empire from 286 to 402 and the Duchy of Milan during the middle and early modern age.

Amsterdam  is the capital and most populous municipality of the Netherlands. Its status as the capital is mandated by the Constitution of the Netherlands, although it is not the seat of the government, which is The Hague. Amsterdam has a population of 851,373 within the city proper, 1,351,587 in the urban area, and 2,410,960 in the Amsterdam metropolitan area. The city is located in the province of North Holland in the west of the country but is not its capital, which is Haarlem. The metropolitan area comprises much of the northern part of the Randstad, one of the larger conurbations in Europe, with a population of approximately 8 million.

Vienna  is the federal capital and largest city of Austria and one of the nine states of Austria. Vienna is Austria's primary city, with a population of about 1.8 million , and its cultural, economic, and political centre. It is the 7th-largest city by population within city limits in the European Union. Until the beginning of the 20th century, it was the largest German-speaking city in the world, and before the splitting of the Austro-Hungarian Empire in World War I, the city had 2 million inhabitants. Today, it has the second largest number of German speakers after Berlin. Vienna is host to many major international organizations, including the United Nations and OPEC. The city is located in the eastern part of Austria and is close to the borders of the Czech Republic, Slovakia, and Hungary. These regions work together in a European Centrope border region. Along with nearby Bratislava, Vienna forms a metropolitan region with 3 million inhabitants. In 2001, the city centre was designated a UNESCO World Heritage Site. In July 2017 it was moved to the list of World Heritage in Danger.

Los Angeles  is the second-most populous city in the United States, after New York City. With an estimated population of almost four million, Los Angeles is the largest and most populous city in the state of California and the cultural, financial, and commercial center of Southern California.

Venice  is a city in northeastern Italy and the capital of the Veneto region. It is situated across a group of 118 small islands that are separated by canals and linked by bridges, of which there are 400. The islands are located in the shallow Venetian Lagoon, an enclosed bay that lies between the mouths of the Po and the Piave rivers . Parts of Venice are renowned for the beauty of their settings, their architecture, and artwork. The lagoon and a part of the city are listed as a UNESCO World Heritage Site.

Lima  is the capital and the largest city of Peru. It is located in the valleys of the Chillón, Rímac and Lurín rivers, in the central coastal part of the country, overlooking the Pacific Ocean. Together with the seaport of Callao, it forms a contiguous urban area known as the Lima Metropolitan Area. With a population of more than 10 million, Lima is the most populous metropolitan area of Peru and the second-largest city in the Americas , behind São Paulo.

Orlando  is a city in the U.S. state of Florida and the county seat of Orange County. Located in Central Florida, it is the center of the Orlando metropolitan area, which had a population of 2,387,138, according to U.S. Census Bureau figures released in March 2016. These figures make it the 24th-largest metropolitan area in the United States, the sixth-largest metropolitan area in the Southern United States, and the third-largest metropolitan area in Florida. As of 2015, Orlando had an estimated city-proper population of 270,934, making it the 73rd-largest city in the United States, the fourth-largest city in Florida, and the state's largest inland city.

Florence  is the capital city of the Italian region of Tuscany. It is the most populous city in Tuscany, with 383,084 inhabitants in 2013, and over 1,520,000 in its metropolitan area.

Budapest  is the capital and the most populous city of Hungary, and one of the largest cities in the European Union. With an estimated 2016 population of 1,752,704 distributed over a land area of about 525 square kilometres , Budapest is also one of the most densely populated major cities in the EU. Budapest is both a city and county, and forms the centre of the Budapest metropolitan area, which has an area of 7,626 square kilometres  and a population of 3,303,786, comprising 33 percent of the population of Hungary. The Budapest metropolitan area had a GDP of $141.0 billion  in 2016, accounting for 49.6 percent of the GDP of Hungary. GDP per capita in the city is $64,283, which means 148% of the EU average measured on purchasing power parity. Thereby the city is among the top 100 GDP performing cities in the world and making it one of the largest regional economies in the European Union.

Berlin  is the capital and the largest city of Germany, as well as one of its 16 constituent states. With a steadily growing population of approximately 3.7 million, Berlin is the second most populous city proper in the European Union behind London and the seventh most populous urban area in the European Union. Located in northeastern Germany on the banks of the rivers Spree and Havel, it is the centre of the Berlin-Brandenburg Metropolitan Region, which has roughly 6 million residents. Due to its location in the European Plain, Berlin is influenced by a temperate seasonal climate. Around one-third of the city's area is composed of forests, parks, gardens, rivers, canals and lakes.

Madrid  is the capital of Spain and the largest municipality in both the Community of Madrid and Spain as a whole. The city has almost 3.2 million inhabitants with a metropolitan area population of approximately 6.5 million. It is the third-largest city in the European Union  after London and Berlin, and its monocentric metropolitan area is the third-largest in the EU after those of London and Paris. The municipality itself covers an area of 604.3 km2 .

Warsaw  is the capital and largest city of Poland. The metropolis stands on the Vistula River in east-central Poland and its population is officially estimated at 1.765 million residents within a greater metropolitan area of 3.101 million residents, which makes Warsaw the 8th most-populous capital city in the European Union. The city limits cover 516.9 square kilometres , while the metropolitan area covers 6,100.43 square kilometres . Warsaw is an alpha global city, a major international tourist destination, and a significant cultural, political and economic hub. With a nominal GDP of $230 billion , it is the wealthiest capital city in Central and Eastern Europe alongside Berlin. Moreover, its historical Old Town was designated a UNESCO World Heritage Site.

Nairobi  is the capital and the largest city of Kenya. The name comes from the Maasai phrase Enkare Nyrobi, which translates to "cool water", a reference to the Nairobi River which flows through the city. The city proper had a population of 3,138,369 in the 2009 census, while the metropolitan area has a population of 6,547,547. The city is popularly referred to as the Green City in the Sun.

Delhi , officially the National Capital Territory of Delhi , is a city and a union territory of India. It is bordered by Haryana on three sides and by Uttar Pradesh to the east. The NCT covers an area of 1,484 square kilometres . According to the 2011 census, Delhi city proper's population was over 11 million, the second-highest in India after Mumbai, while the whole NCT's population was about 16.8 million. Delhi's urban area is now considered to extend beyond the NCT boundary and include the neighboring cities of Faridabad, Gurgaon, Noida and Ghaziabad in an area now called as Central National Capital Region  with an estimated 2016 population of over 26 million people, making it the world's second-largest urban area according to United Nations. As of 2016 recent estimates of the metro economy of its urban area have ranked Delhi either the most or second-most productive metro area of India. Delhi is the second-wealthiest city after Mumbai in India, with a total wealth of $450 billion and home to 18 billionaires and 23,000 millionaires.

Mexico City, or the City of Mexico , is the capital of Mexico and the most populous city in North America. Mexico City is one of the most important cultural and financial centers in the Americas. It is located in the Valley of Mexico , a large valley in the high plateaus in the center of Mexico, at an altitude of 2,240 metres . The city has 16 boroughs.

Dublin  is the capital of and largest city in Ireland. Dublin is located in the province of Leinster on the east coast of Ireland, at the mouth of the River Liffey and bordered on the South by the Wicklow Mountains. The city has an urban area population of 1,173,179. The population of the Dublin Region, as of 2016, was 1,347,359 people, and the population of the Greater Dublin area was 1,904,806.

San Francisco , officially the City and County of San Francisco, is the cultural, commercial, and financial center of Northern California. It covers an area of about 46.89 square miles , mostly at the north end of the San Francisco Peninsula in the San Francisco Bay Area. San Francisco is the fourth-most populous city in California and the 13th-most populous in the United States, with a 2017 census-estimated population of 884,363. The consolidated city-county is also the fifth most densely populated U.S. county, behind only four of the five New York City boroughs. As of 2016, it was the 7th highest-income county in the United States, with a per capita personal income of $110,418.

Hangzhou  formerly romanized as Hangchow, is the capital and most populous city of Zhejiang Province in east China. It sits at the head of Hangzhou Bay, which separates Shanghai and Ningbo. Hangzhou grew to prominence as the southern terminus of the Grand Canal and has been one of the most renowned and prosperous cities in China for much of the last millennium. The city's West Lake, a UNESCO World Heritage Site, immediately west of the city, is amongst its best-known attractions.

Denpasar   is the capital of Bali and the main gateway to the island. The city is also a hub for other cities in the Lesser Sunda Islands.

Brussels , officially the Brussels-Capital Region , is a region of Belgium comprising 19 municipalities, including the City of Brussels, which is the de jure capital of Belgium. The Brussels-Capital Region is located in the central portion of the country and is a part of both the French Community of Belgium and the Flemish Community, but is separate from Flanders  and Wallonia. It covers 161 km2 , a relatively small area compared to the two other regions, and has a population of 1.2 million, with a metropolitan area of over 2.1 million people, the largest in Belgium. It is also part of a large conurbation extending towards Ghent, Antwerp, Leuven and Walloon Brabant, home to over 5 million people.

Burgas , sometimes transliterated as Bourgas, is the second largest city on the Bulgarian Black Sea Coast and the fourth-largest in Bulgaria after Sofia, Plovdiv, and Varna, with a population of 211,033 inhabitants, while 277,922 live in its urban area. It is the capital of Burgas Province and an important industrial, transport, cultural and tourist centre.

Munich  is the capital and the most populated city in the German state of Bavaria, on the banks of the River Isar north of the Bavarian Alps. Munich is also the third largest city in Germany, after Berlin and Hamburg, and the 12th largest city in the European Union, with a population of around 1.5 million. The Munich Metropolitan Region is home to 6 million people.

Zhuhai  is a prefecture-level city on the southern coast of Guangdong province in China. Located in the Pearl River Delta, Zhuhai borders Jiangmen to the northwest, Zhongshan to the north, and Macau to the south. Zhuhai was one of the original Special Economic Zones established in the 1980s. Zhuhai is also one of China's premier tourist destinations, being called the Chinese Riviera. While the city is located in the traditionally Cantonese-speaking region of Guangdong Province, a significant portion of population is now made up of Mandarin speaking economic migrants originally from inland provinces.

Lisbon  is the capital and the largest city of Portugal, with an estimated population of 552,700 within its administrative limits in an area of 100.05 km2. Its urban area extends beyond the city's administrative limits with a population of around 2.7 million people, being the 11th-most populous urban area in the European Union. About 3 million people live in the Lisbon Metropolitan Area . It is Europe's westernmost capital city and the only one along the Atlantic coast. Lisbon lies in the western Iberian Peninsula on the Atlantic Ocean and the River Tagus. The westernmost areas of its metro area form the westernmost point of Continental Europe, which is known as Cabo da Roca, located in the Sintra Mountains

Djerba , also transliterated as Jerba or Jarbah, is, at 514 square kilometres , the largest island of North Africa, located in the Gulf of Gabès, off the coast of Tunisia. It had a population of 139,544 at the 2004 Census, while the latest official estimate  is 163,726. Citing the long and unique Jewish minority's history on Djerba, Tunisia has sought UNESCO World Heritage Status protections for the island.

Agra  is a city on the banks of the river Yamuna in the northern state of Uttar Pradesh, India. It is 378 kilometres  west of the state capital, Lucknow, 206 kilometres  south of the national capital New Delhi, 58 kilometres  south of Mathura and 125 kilometres  north of Gwalior. Agra is one of the most populous cities in Uttar Pradesh, and the 24th most populous in India.

Kraków , also spelled Cracow or Krakow, is the second largest and one of the oldest cities in Poland. Situated on the Vistula River  in the Lesser Poland  region, the city dates back to the 7th century. Kraków has traditionally been one of the leading centres of Polish academic, cultural and artistic life and is one of Poland's most important economic hubs. It was the capital of the Crown of the Kingdom of Poland from 1038 to 1569; the Polish–Lithuanian Commonwealth from 1569 to 1596, the Free City of Kraków from 1815 to 1846; the Grand Duchy of Cracow from 1846 to 1918; and Kraków Voivodeship from the 14th century to 1998. It has been the capital of Lesser Poland Voivodeship since 1999.

Sydney  is the state capital of New South Wales and the most populous city in Australia and Oceania. Located on Australia's east coast, the metropolis surrounds Port Jackson and sprawls about 70 km  on its periphery towards the Blue Mountains to the west, Hawkesbury to the north and Macarthur to the south. Sydney is made up of 658 suburbs, 40 local government areas and 15 contiguous regions. Residents of the city are known as "Sydneysiders". As of June 2017 Sydney's estimated population was 5,131,326.

Rockhampton is a city in the Rockhampton Region, Queensland, Australia. The estimated urban population of Rockhampton in June 2015 was 80,665, making it the fourth largest city in the state outside of the cities of South East Queensland. and the twenty-second largest city in Australia.

Baghdad  is the capital of Iraq. The population of Baghdad, as of 2016, is approximately 8,765,000, making it the largest city in Iraq, the second largest city in the Arab world , and the second largest city in Western Asia .

Multan , is a Pakistani city and the headquarters of Multan District in the province of Punjab. Located on the banks of the Chenab River, Multan is Pakistan's seventh most populous city, and is the major cultural and economic centre of southern Punjab.

Luanda, formerly named São Paulo da Assunção de Loanda, is the capital and largest city in Angola, and the country's most populous and important city, primary port and major industrial, cultural and urban centre. Located on Angola's coast with the Atlantic Ocean, Luanda is both Angola's chief seaport and its administrative centre. It is also the capital city of Luanda Province, and the world's third most populous Portuguese-speaking city, behind the Brazilian cities of São Paulo and Rio de Janeiro, and the most populous Portuguese-speaking capital city in the world, ahead of Brasília, Maputo and Lisbon.

Harbin  is the capital of Heilongjiang province, and largest city in the northeastern region of the People's Republic of China. Holding sub-provincial administrative status, Harbin has direct jurisdiction over nine metropolitan districts, two county-level cities and seven counties. Harbin is the eighth most populous Chinese city according to the 2010 census, the built-up area made of seven out of nine urban districts  had 5,282,093 inhabitants, while the total population of the sub-provincial city was up to 10,635,971. Harbin serves as a key political, economic, scientific, cultural, and communications hub in Northeast China, as well as an important industrial base of the nation.

Foshan, formerly romanized as Fatshan, is a prefecture-level city in central Guangdong Province in southeastern China. The entire prefecture covers 3,848.49 km2  and has an urban population around 7.2 million. It forms part of the western side of the Pearl River Delta Economic Zone, which includes Guangzhou to the north and Shenzhen to the east.

Chengdu , formerly romanized as Chengtu, is a sub-provincial city which serves as the capital of China's Sichuan province. It is one of the three most populous cities in Western China, the other two being Chongqing and Xi'an. As of 2014, the administrative area houses 14,427,500 inhabitants, with an urban population of 10,152,632. At the time of the 2010 census, Chengdu was the 5th-most populous agglomeration in China, with 10,484,996 inhabitants in the built-up area including Xinjin County and Deyang's Guanghan City.

Nanjing , formerly romanized as Nanking and Nankin, is the capital of Jiangsu province of the People's Republic of China and the second largest city in the East China region, with an administrative area of 6,600 km2  and a total population of 8,270,500 as of 2016. The inner area of Nanjing enclosed by the city wall is Nanjing City , with an area of 55 km2 , while the Nanjing Metropolitan Region includes surrounding cities and areas, covering over 60,000 km2 , with a population of over 30 million.

Athens  is the capital and largest city of Greece. Athens dominates the Attica region and is one of the world's oldest cities, with its recorded history spanning over 3,400 years and its earliest human presence starting somewhere between the 11th and 7th millennium BC.

Lagos   is a city in the Nigerian state of Lagos. The city, with its adjoining conurbation, is the most populous in Nigeria, and the most populous on the African continent. It is one of the fastest growing cities in the world, and also one of the most populous urban agglomerations. Lagos is a major financial centre in Africa; the megacity has the highest GDP, and also houses one of the largest and busiest ports on the continent.

Cape Town  is a coastal city in South Africa. It is the second-most populous urban area in South Africa after Johannesburg. It is the capital and primate city of the Western Cape province.

Giza  is the third-largest city in Egypt and the capital of the Giza Governorate. It is located on the west bank of the Nile, 4.9 km  southwest of central Cairo. Along with Cairo Governorate, Shubra El-Kheima, Helwan, 6th October City and Obour, the five form Greater Cairo metropolis.

Kanpur  is the 12th most populous city in India and the second largest city in the state of Uttar Pradesh after Lucknow. The city is the administrative headquarters of Kanpur Nagar district and Kanpur division.

Osaka   is a designated city in the Kansai region of Japan. It is the capital city of Osaka Prefecture and the largest component of the Keihanshin Metropolitan Area, the second largest metropolitan area in Japan and among the largest in the world with over 19 million inhabitants. Situated at the mouth of the Yodo River on Osaka Bay, Osaka is the second largest city in Japan by daytime population after Tokyo's 23 wards and the third largest city by nighttime population after Tokyo's 23 wards and Yokohama, serving as a major economic hub for the country.

Casablanca , located in the central-western part of Morocco bordering the Atlantic Ocean, is the largest city in Morocco. It is also the largest city in the Maghreb, as well as one of the largest and most important cities in Africa, both economically and demographically.

Nanning  is the capital of the Guangxi Zhuang Autonomous Region in southern China. It is known as the "Green City" because of its abundance of lush subtropical foliage. As of 2014 it had a population of 6,913,800 with 4,037,000 in its urban area.

Yangon  was the capital of the Yangon Region of Myanmar, also known as Burma. Yangon served as the capital of Myanmar until 2006, when the military government relocated the capital to the purpose-built city of Naypyidaw in central Myanmar. With over 7 million people, Yangon is Myanmar's largest city and its most important commercial centre.

Zibo  is a prefecture-level city in central Shandong province, China. It borders the provincial capital of Jinan to the west, Laiwu and Tai'an to the southwest, Linyi to the south, Weifang to the east, Dongying to the northeast, and Binzhou to the north.

Aleppo  is a city in Syria, serving as the capital of the Aleppo Governorate, the most populous Syrian governorate. With an official population of 4.6 million in 2010, Aleppo was the largest Syrian city before the Syrian Civil War; however, now Aleppo is probably the second-largest city in Syria after the capital Damascus.

Havana  is the capital city, largest city, province, major port, and leading commercial center of Cuba. The city has a population of 2.1 million inhabitants, and it spans a total of 781.58 km3  – making it the largest city by area, the most populous city, and the fourth largest metropolitan area in the Caribbean region. The city extends mostly westward and southward from the bay, which is entered through a narrow inlet and which divides into three main harbors: Mari melena, Guanabacoa and Antares. The sluggish Almendares River traverses the city from south to north, entering the Straits of Florida a few miles west of the bay.

Medan  is the capital of North Sumatra province in Indonesia. Located along the northeastern coast of Sumatra island, Medan is the fourth biggest city by population in Indonesia, behind Jakarta, Surabaya and Bandung. With 2,097,610 inhabitants at the 2010 census, Medan remains the largest settlement outside Java island and enjoys a diversity of multicultural peoples. Bordered by the Strait of Malacca, Medan is a busy trading city around the island as located near the strait which is one of the most important shipping lanes in the world. Medan is the gateway to the western part of Indonesia, accessible via the Port of Belawan and Kuala Namu International Airport, stated the city as the third largest city in the country by economy after Jakarta and Surabaya, this city economy is linked well with Malaysian cities  and Singapore by trade, service and natural resource exchanges. Both the seaport and the airport are connected to the city center via toll road and railway. Medan also became the first city in Indonesia to have an airport supported with train service.

Perth  is the capital and largest city of the Australian state of Western Australia. It is the fourth-most populous city in Australia, with a population of 2,022,044 living in Greater Perth. Perth is part of the South West Land Division of Western Australia, with the majority of the metropolitan area located on the Swan Coastal Plain, a narrow strip between the Indian Ocean and the Darling Scarp. The first areas settled were on the Swan River at Guildford, with the city's central business district and port  both later founded downriver.

Yantai, formerly known as Zhifu or Chefoo, is a prefecture-level city on the Bohai Strait in northeastern Shandong Province, China. Lying on the southern coast of the Korea Bay, Yantai borders Qingdao on the southwest and Weihai on the east. It is the largest fishing seaport in Shandong. Its population was 6,968,202 during the 2010 census, of whom 2,227,733 lived in the built-up area made up of the 4 urban districts of Zhifu, Muping, Fushan, and Laishan.

Agra  is a city on the banks of the river Yamuna in the northern state of Uttar Pradesh, India. It is 378 kilometres  west of the state capital, Lucknow, 206 kilometres  south of the national capital New Delhi, 58 kilometres  south of Mathura and 125 kilometres  north of Gwalior. Agra is one of the most populous cities in Uttar Pradesh, and the 24th most populous in India.

Vienna  is the federal capital and largest city of Austria and one of the nine states of Austria. Vienna is Austria's primary city, with a population of about 1.8 million , and its cultural, economic, and political centre. It is the 7th-largest city by population within city limits in the European Union. Until the beginning of the 20th century, it was the largest German-speaking city in the world, and before the splitting of the Austro-Hungarian Empire in World War I, the city had 2 million inhabitants. Today, it has the second largest number of German speakers after Berlin. Vienna is host to many major international organizations, including the United Nations and OPEC. The city is located in the eastern part of Austria and is close to the borders of the Czech Republic, Slovakia, and Hungary. These regions work together in a European Centrope border region. Along with nearby Bratislava, Vienna forms a metropolitan region with 3 million inhabitants. In 2001, the city centre was designated a UNESCO World Heritage Site. In July 2017 it was moved to the list of World Heritage in Danger.

Depok  is a city in West Java province, Indonesia on the southern border of Jakarta SCR in the Greater Jakarta metropolitan region. The "de" in Jabodetabek refers to Depok, while Depok word is an acronym of De Eerste Protestants Onderdaan Kerk . There is also a saying that the word "depok" itself comes from Sundanese meaning hermitage or abode of one living in seclusion.

Montreal  is the most populous municipality in the Canadian province of Quebec and the second-most populous municipality in Canada as a whole, after Toronto. Originally called Ville-Marie, or "City of Mary", it is named after Mount Royal, the triple-peaked hill in the heart of the city. The city is centred on the Island of Montreal, which took its name from the same source as the city, and a few much smaller peripheral islands, the largest of which is Île Bizard. It has a distinct four-season continental climate with warm to hot summers and cold, snowy winters.

Warsaw  is the capital and largest city of Poland. The metropolis stands on the Vistula River in east-central Poland and its population is officially estimated at 1.765 million residents within a greater metropolitan area of 3.101 million residents, which makes Warsaw the 8th most-populous capital city in the European Union. The city limits cover 516.9 square kilometres , while the metropolitan area covers 6,100.43 square kilometres . Warsaw is an alpha global city, a major international tourist destination, and a significant cultural, political and economic hub. With a nominal GDP of $230 billion , it is the wealthiest capital city in Central and Eastern Europe alongside Berlin. Moreover, its historical Old Town was designated a UNESCO World Heritage Site.

Nantong  is a prefecture-level city in Jiangsu province, China. Located on the northern bank of the Yangtze River, near the river mouth. Nantong is a vital river port bordering Yancheng to the north, Taizhou to the west, Suzhou and Shanghai to the south across the river, and the East China Sea to the east. Its current population is 7,282,835 at the 2010 census, 1,994,708 of whom live in the built-up area made up of three urban districts.

Philadelphia  is the largest city in the U.S. state and Commonwealth of Pennsylvania, and the sixth-most populous U.S. city, with a 2017 census-estimated population of 1,580,863. Since 1854, the city has been coterminous with Philadelphia County, the most populous county in Pennsylvania and the urban core of the eighth-largest U.S. metropolitan statistical area, with over 6 million residents as of 2017. Philadelphia is also the economic and cultural anchor of the greater Delaware Valley, located along the lower Delaware and Schuylkill Rivers, within the Northeast megalopolis. The Delaware Valley's population of 7.2 million ranks it as the eighth-largest combined statistical area in the United States.

Luoyang, formerly romanized as Loyang, is a city located in the confluence area of Luo River and Yellow River in the west of Henan province. Governed as a prefecture-level city, it borders the provincial capital of Zhengzhou to the east, Pingdingshan to the southeast, Nanyang to the south, Sanmenxia to the west, Jiyuan to the north, and Jiaozuo to the northeast. As of the final 2010 census, Luoyang had a population of 6,549,941 inhabitants with 1,857,003 people living in the built-up  area made of the city's five urban districts, all of which except the Jili District are not urbanized yet.

Linyi  is a prefecture-level city in the south of Shandong province, China. As of 2011, Linyi is the largest prefecture-level city in Shandong, both by area and population, Linyi borders Rizhao to the east, Weifang to the northeast, Zibo to the north, Tai'an to the northwest, Jining to the west, Zaozhuang to the southwest, and the province of Jiangsu to the south. The name of the city Linyi  literally means "close to the Yi River".

Kyoto , literally meaning "Capital City", is the capital city of Kyoto Prefecture, located in the Kansai region of Japan. It is most well known in Japanese history for being the former Imperial capital of Japan for more than one thousand years, as well as a major part of the Kyoto-Osaka-Kobe metropolitan area.

Stockholm  is the capital of Sweden and the most populous city in the Nordic countries; 952,058 people live in the municipality, approximately 1.5 million in the urban area, and 2.3 million in the metropolitan area. The city stretches across fourteen islands where Lake Mälaren flows into the Baltic Sea. Just outside the city and along the coast is the island chain of the Stockholm archipelago. The area has been settled since the Stone Age, in the 6th millennium BC, and was founded as a city in 1252 by Swedish statesman Birger Jarl. It is also the capital of Stockholm County.

San Diego  is a major city in California, United States. It is in San Diego County, on the coast of the Pacific Ocean in Southern California, approximately 120 miles  south of Los Angeles and immediately adjacent to the border with Mexico.

Quito  , formally San Francisco de Quito, is the capital city of Ecuador, and at an elevation of 2,850 metres  above sea level, it is the second-highest official capital city in the world, after La Paz, and the one which is closest to the equator. It is located in the Guayllabamba river basin, on the eastern slopes of Pichincha, an active stratovolcano in the Andes mountains. With a population of 2,671,191 according to the last census , Quito is the second most populous city in Ecuador, after Guayaquil. It is also the capital of the Pichincha province and the seat of the Metropolitan District of Quito. The canton recorded a population of 2,239,191 residents in the 2010 national census. In 2008, the city was designated as the headquarters of the Union of South American Nations.

Daqing  is a prefecture-level city in the west of Heilongjiang province, People's Republic of China. The name literally means "Great Celebration". Daqing is known as the Oil Capital of China and has experienced a phenomenal boom since oil was discovered at the Daqing Oil Field in 1959.

Copenhagen  is the capital and most populous city of Denmark. The city has a population of 775,033 , of whom 613,288 live in the Municipality of Copenhagen and the Copenhagen urban area has a population of 1,308,893 . Copenhagen is situated on the eastern coast of the island of Zealand; another small portion of the city is located on Amager, and is separated from Malmö, Sweden, by the strait of Øresund. The Øresund Bridge connects the two cities by rail and road.

Multan , is a Pakistani city and the headquarters of Multan District in the province of Punjab. Located on the banks of the Chenab River, Multan is Pakistan's seventh most populous city, and is the major cultural and economic centre of southern Punjab.

Davao City, officially the City of Davao , is a highly urbanized city in the island of Mindanao, Philippines. The city has a total land area of 2,443.61 km2 , and has a population of 1,632,991 based on the 2015 census, making the city as the largest city in the Philippines in terms of land area, and the most populous city in the country outside Metro Manila. The city is also the third-most-populous city in the Philippines and the most populous in Mindanao.

Belgrade  is the capital and largest city of Serbia. It is located at the confluence of the Sava and Danube rivers, where the Pannonian Plain meets the Balkans. The urban area of the City of Belgrade has a population of 1.23 million, while nearly 1.7 million people live within its administrative limits.

Konya  is a major city in south-western edge of the Central Anatolian Plateau and is the seventh-most-populous city in Turkey with a metropolitan population of over 2.1 million.  Konya is an economically and industrially developed city and the capital of Konya Province.

Wuhu  is a prefecture-level city in southeastern Anhui province, China. Sitting on the southeast bank of the Yangtze River, Wuhu borders Xuancheng to the southeast, Chizhou and Tongling to the southwest, Hefei city to the northwest, Ma'anshan city to the northeast, Jiangsu Province to the east, and is approximately 90 km  southwest of Nanjing. As of 2017, the city had a population of approximately 3,696,000 officially registered inhabitants.

Antalya  is the fifth-most populous city in Turkey and the capital of its eponymous province. Located on Anatolia's flourishing southwest coast bordered by the Taurus Mountains, Antalya is the largest Turkish city on the Mediterranean coast with over one million people in its metropolitan area.

Belgrade  is the capital and largest city of Serbia. It is located at the confluence of the Sava and Danube rivers, where the Pannonian Plain meets the Balkans. The urban area of the City of Belgrade has a population of 1.23 million, while nearly 1.7 million people live within its administrative limits.

Ufa  is the capital city of the Republic of Bashkortostan, Russia, and the industrial, economic, scientific and cultural center of the republic. As of the 2010 Census, its population was 1,062,319, making it the eleventh most populous city in Russia.

Tripoli  is the capital city and the largest city of Libya, with a population of about 1.1 million people in 2015. It is located in the northwest of Libya on the edge of the desert, on a point of rocky land projecting into the Mediterranean Sea and forming a bay. It includes the port of Tripoli and the country's largest commercial and manufacturing centre. It is also the site of the University of Tripoli. The vast Bab al-Azizia barracks, which includes the former family estate of Muammar Gaddafi, is also located in the city. Colonel Gaddafi largely ruled the country from his residence in this barracks.

Cologne  is the largest city in the German federal state of North Rhine-Westphalia and the fourth most populated city in Germany . It is located within the Rhine-Ruhr metropolitan region which is Germany's largest and one of Europe's major metropolitan areas. Cologne is about 45 kilometres  southeast of North Rhine-Westphalia's capital of Düsseldorf and 25 kilometres  northwest of Bonn.

Naples  is the regional capital of Campania and the third-largest municipality in Italy after Rome and Milan. In 2017, around 967,069 people lived within the city's administrative limits while its province-level municipality has a population of 3,115,320 residents. Its continuously built-up urban area  is the second largest in Italy after Milan and one of the largest metropolises on the Tyrrhenian Sea , with a population of between 4.4 and 5.3 milion.

Kathmandu  is the capital city of the Federal Democratic Republic of Nepal. It is the largest metropolis in Nepal, with a population of 1.5 million in the city proper, and 3 million in its urban agglomeration across the Kathmandu Valley, which includes the towns of Lalitpur, Kirtipur, Madhyapur Thimi, Bhaktapur making the total population to roughly 5 million people and the municipalities across Kathmandu valley. Kathmandu is also the largest metropolis in the Himalayan hill region.

Sfax  is a city in Tunisia, located 270 km  southeast of Tunis. The city, founded in AD 849 on the ruins of Taparura and Thaenae, is the capital of the Sfax Governorate , and a Mediterranean port. Sfax has a population of 330,440 . The main economic activities of Sfax are industries , agriculture , fishing  and trade . The city is often described as Tunisia's "second city" .

Bogor  is a city in the West Java province, Indonesia. Located around 60 kilometers  south from the national capital of Jakarta, Bogor is the 6th largest city of Jabodetabek  and the 14th nationwide. The city covers an area of 118.5 km2, and it had a population of 950,334 at the 2010 Census; the latest official estimate  was 1,030,720. Bogor is an important economic, scientific, cultural and tourist center, as well as a mountain resort.

Fez  is a city in northern inland Morocco and the capital of the Fas-Meknas administrative region. It is the second or third largest city of Morocco after Casablanca, with a population of 1.1 million . Located to the northeast of Atlas Mountains, Fez is situated at the crossroad of the important cities of all regions; 206 km  from Tangier to the northwest, 246 km  from Casablanca, 169 km  from Rabat to the west, and 387 km  from Marrakesh to the southwest which leads to the Trans-Saharan trade route. It is surrounded by the high grounds, and the old city is penetrated by the River of Fez flowing from the west to east.

Mombasa  is a city on the coast of Kenya. It is the country's oldest  and second-largest city, after the capital Nairobi, with an estimated population of about 1.2 million people in 2016. Its metropolitan region is the second largest in the country and has a population of approximately two million people. Administratively, Mombasa is the county seat of Mombasa County.

Ottawa  is the capital city of Canada. It stands on the south bank of the Ottawa River in the eastern portion of southern Ontario. Ottawa borders Gatineau, Quebec; the two form the core of the Ottawa–Gatineau census metropolitan area  and the National Capital Region . As of 2016, Ottawa had a city population of 934,243 and a metropolitan population of 1,323,783 making it the fourth-largest city and the fifth-largest CMA in Canada.

Tunis  is the capital and the largest city of Tunisia. The greater metropolitan area of Tunis, often referred to as Grand Tunis, has some 2,700,000 inhabitants.

Calgary  is a city in the Canadian province of Alberta. It is situated at the confluence of the Bow River and the Elbow River in the south of the province, in an area of foothills and prairie, about 80 km  east of the front ranges of the Canadian Rockies. The city anchors the south end of what Statistics Canada defines as the "Calgary–Edmonton Corridor".

Ufa  is the capital city of the Republic of Bashkortostan, Russia, and the industrial, economic, scientific and cultural center of the republic. As of the 2010 Census, its population was 1,062,319, making it the eleventh most populous city in Russia.

Turin  is a city and an important business and cultural centre in northern Italy. It is the capital city of the Metropolitan City of Turin  and of the Piedmont region, and was the first capital city of Italy from 1861 to 1865. The city is located mainly on the western bank of the Po River, in front of Susa Valley, and is surrounded by the western Alpine arch and Superga Hill. The population of the city proper is 883 281  while the population of the urban area is estimated by Eurostat to be 1.7 million inhabitants. The Turin metropolitan area is estimated by the OECD to have a population of 2.2 million.

Cebu  is a province of the Philippines located in the Central Visayas  region, and consisting of a main island and 167 surrounding islands and islets. Its capital is Cebu City, the oldest city and first capital of the Philippines, which is politically independent from the provincial government. The Cebu Metropolitan Area is formed by seven neighboring cities  and eight other local government units. Mactan–Cebu International Airport, located on Mactan Island, is the second busiest airport in the Philippines.

Aligarh  is a city in the Northern Indian state of Uttar Pradesh that is famous for lock industries and the administrative headquarters of the Aligarh district. It lies 307 kilometres  northwest of Kanpur and is approximately 145 kilometres  southeast of the capital, New Delhi. Notable as the seat of Aligarh Muslim University, Aligarh is one of the largest cities in Uttar Pradesh, and is the 55th largest city in India.

Marseille , is the second-largest city of France and the largest city of the Provence historical region. The capital of the Bouches-du-Rhône department and Provence-Alpes-Côte d'Azur administrative region, it is located on France's south coast and had a population of 852,516 in 2012, and an area of 241 km2 , the third-largest metropolitan area in France after Paris and Lyon.

Jacksonville is the most populous city in the U.S. state of Florida and the largest city by area in the contiguous United States. It is the seat of Duval County, with which the city government consolidated in 1968. Consolidation gave Jacksonville its great size and placed most of its metropolitan population within the city limits. With an estimated population of 892,062 as of 2017, Jacksonville is also the most populous city in the southeastern United States. The Jacksonville metropolitan area has a population of 1,631,488 and is the fourth largest in Florida.

Indianapolis  is the capital and most populous city of the U.S. state of Indiana and the seat of Marion County. As of 2017, Indianapolis is the third most populous city in the American Midwest and 16th most populous in the U.S., with an estimated population of 863,002. The Indianapolis metropolitan area is the 34th most populous metropolitan statistical area in the U.S., with 2,028,614 residents. Its combined statistical area ranks 27th, with a population of 2,411,086. Indianapolis covers 368 square miles , making it the 16th largest city by land area in the U.S.

Valencia , officially València , on the east coast of Spain, is the capital of the autonomous community of Valencia and the third-largest city in Spain after Madrid and Barcelona, with around 800,000 inhabitants in the administrative centre. Its urban area extends beyond the administrative city limits with a population of around 1.5–1.6 million people. Valencia is Spain's third largest metropolitan area, with a population ranging from 1.7 to 2.5 million depending on how the metropolitan area is defined. The Port of Valencia is the 5th busiest container port in Europe and the busiest container port on the Mediterranean Sea. The city is ranked at Gamma+ in the Globalization and World Cities Research Network. Valencia is integrated into an industrial area on the Costa del Azahar .

Glasgow  is the largest city in Scotland, and third most populous in the United Kingdom. Historically part of Lanarkshire, the city now forms the Glasgow City council area, one of the 32 council areas of Scotland; the local authority is Glasgow City Council. Glasgow is situated on the River Clyde in the country's West Central Lowlands. Inhabitants of the city are referred to as "Glaswegians" or "Weegies". It is the fourth most visited city in the UK.

Abuja  is the capital city of Nigeria located in the centre of the country within the Federal Capital Territory . It is a planned city and was built mainly in the 1980s, replacing the country's most populous city of Lagos as the capital on 12 December 1991. Abuja's geography is defined by Aso Rock, a 400-metre  monolith left by water erosion. The Presidential Complex, National Assembly, Supreme Court and much of the city extend to the south of the rock. Zuma Rock, a 792-metre  monolith, lies just north of the city on the expressway to Kaduna.

Charlotte  is the most populous city in the U.S. state of North Carolina. Located in the Piedmont, it is the county seat of Mecklenburg County.

The Daily Wahdat or Wahdat daily  is a Pashto daily newspaper published in Peshawar, in the Khyber Pakhtunkhwa of Pakistan.

Dushanbe  is the capital and largest city of Tajikistan. Dushanbe means Monday in the Tajik language. It was named this way because it grew from a village that originally had a popular market on Mondays. As of 2016, Dushanbe had a population of 802,700.

Leeds   is a city in the metropolitan borough of Leeds, in the county of West Yorkshire, England. The city lies within the United Kingdom's fourth-most populous urban area, with a population of 2.6 million..

Astana  is the capital city of Kazakhstan. It is located on the banks of the Ishim River in the north portion of Kazakhstan, within the Aqmola Region, though administered separately from the region as a city with special status. The 2017 official estimate reported a population of 1,029,556 within the city limits, making it the second largest city in Kazakhstan, behind Almaty.

Mysore , officially Mysuru, is the third most populous city in the state of Karnataka, India. It is also the second largest urban agglomeration in Karnataka in population. It is located in the foothills of the Chamundi Hills about 146 km  southwest of Bangalore and spread across an area of 152 km2 . The estimated population is 1,014,227 as of 2017. Mysore City Corporation is responsible for the civic administration of the city, which is also the headquarters of the Mysore district and the Mysore division.

Suzhou , formerly romanized as Soochow, is a major city located in southeastern Jiangsu Province of East China, about 100 km  northwest of Shanghai. It is a major economic center and focal point of trade and commerce, and the second largest city in the province, after its capital Nanjing. The city is situated on the lower reaches of the Yangtze River and the shores of Lake Tai and belongs to the Yangtze River Delta region. Administratively, Suzhou is a prefecture-level city with a population of 4.33 million in its city proper, and a total resident population  of 10.58 million in its administrative area. Its urban population grew at an unprecedented rate of 6.5% between 2000 and 2014, which is the highest among cities with more than 5 million people.

Fort Worth is the 15th-largest city in the United States and the fifth-largest city in the state of Texas. The city is located in North Texas. It is the county seat of Tarrant County, covering nearly 350 square miles  into four other counties: Denton, Johnson, Parker and Wise. According to the 2017 census estimates, Fort Worth's population is 874,168. The city is the second-largest in the Dallas–Fort Worth–Arlington metropolitan area , 4th most populous metropolitan area in the United States.

Seville  is the capital and largest city of the autonomous community of Andalusia and the province of Seville, Spain. It is situated on the plain of the river Guadalquivir. The inhabitants of the city are known as sevillanos  or hispalenses, after the Roman name of the city, Hispalis. Seville has a municipal population of about 703,000 as of 2011, and a metropolitan population of about 1.5 million, making it the fourth-largest city in Spain and the 30th most populous municipality in the European Union. Its Old Town, with an area of 4 square kilometres , contains three UNESCO World Heritage Sites: the Alcázar palace complex, the Cathedral and the General Archive of the Indies. The Seville harbour, located about 80 kilometres  from the Atlantic Ocean, is the only river port in Spain. Seville is also the hottest major metropolitan area in the geographical Western Europe, with summer average high temperatures of above 35 °C .

Gorakhpur is a city located along the banks of Rapti river in the north-eastern part of the Indian state of Uttar Pradesh, with a population of 673,446. It is located near the Nepal border, 273 kilometres east of the state capital Lucknow. It is the administrative headquarters of Gorakhpur district and Gorakhpur division. The city is home to the Gorakhnath Math, a Gorakshanath temple.

Antipolo, officially the City of Antipolo, , or simply known as Antipolo City, is a 1st class city and the capital of the province of Rizal, Philippines. According to the 2015 census, it has a population of 776,386 people.

Pasig is a city in the Philippines. Located along the eastern border of Metro Manila, Pasig is bordered on the west by Quezon City and Mandaluyong; to the north by Marikina; to the south by Makati, Pateros, and Taguig; and to the east by Antipolo, the municipality of Cainta and Taytay in the province of Rizal.

Pretoria is a city in the northern part of Gauteng, South Africa. It is one of the country's three capital cities, serving as the seat of the executive branch of government . Pretoria has a reputation for being an academic city with three universities and the Council for Scientific and Industrial Research  located in its eastern suburbs, the city also hosts the South African Bureau of Standards making the city a hub for research. Pretoria is the central part of the Tshwane Metropolitan Municipality which was formed by the amalgamation of several former local authorities including Centurion and Soshanguve. There have been proposals to change the name of Pretoria itself to Tshwane, and the proposed name change has caused some controversy.

Kananga, formerly known as Luluabourg or Luluaburg, is the capital city of the Lulua Province in the Democratic Republic of the Congo and was the capital of the former Kasaï-Occidental Province. The city has an estimated population of 1,463,556.

Colombo  is the commercial capital and largest city of Sri Lanka. According to the Brookings Institution, Colombo metropolitan area has a population of 5.6 million, and 752,993 in the city proper. It is the financial centre of the island and a popular tourist destination. It is located on the west coast of the island and adjacent to the Greater Colombo area which includes Sri Jayawardenepura Kotte, the legislative capital of Sri Lanka and Dehiwala-Mount Lavinia. Colombo is often referred to as the capital since Sri Jayawardenepura Kotte is within the urban area of, and a suburb of, Colombo. It is also the administrative capital of the Western Province and the district capital of Colombo District. Colombo is a busy and vibrant place with a mixture of modern life and colonial buildings and ruins. It was the legislative capital of Sri Lanka until 1982.

Noida, short for the New Okhla Industrial Development Authority, is a systematically planned Indian city under the management of the New Okhla Industrial Development Authority . Being a satellite city of Delhi, it is part of the National Capital Region of India. As per provisional reports of Census of India, the population of Noida in 2011 was 642,381. Noida is located in Gautam Buddh Nagar district of Uttar Pradesh state in close proximity to NCT of Delhi. The district's administrative headquarters are in the nearby town of Greater Noida. However, the district's highest government official, the District Magistrate , has its official camp office in Noida. The city is a part of the Noida Vidhan Sabha  constituency and Gautam Buddha Nagar . Minister of State for Culture, Tourism of Civil Aviation Mahesh Sharma of the BJP is the current MP of Noida.

Denver , officially the City and County of Denver, is the capital and most populous municipality of the U.S. state of Colorado. Denver is in the South Platte River Valley on the western edge of the High Plains just east of the Front Range of the Rocky Mountains. The Denver downtown district is immediately east of the confluence of Cherry Creek with the South Platte River, approximately 12 mi  east of the foothills of the Rocky Mountains. Denver is nicknamed the Mile High City because its official elevation is exactly one mile  above sea level, making it the highest major city in the United States. The 105th meridian west of Greenwich, the longitudinal reference for the Mountain Time Zone, passes directly through Denver Union Station.

Seattle  is a seaport city on the west coast of the United States. It is the seat of King County, Washington. With an estimated 725,000 residents as of 2018, Seattle is the largest city in both the state of Washington and the Pacific Northwest region of North America. According to U.S. Census data released in 2018, the Seattle metro area’s population stands at 3.87 million, and ranks as the 15th-largest in the United States. In July 2013, it was the fastest-growing major city in the United States and remained in the Top 5 in May 2015 with an annual growth rate of 2.1%. In July 2016, Seattle was again the fastest-growing major U.S. city, with a 3.1% annual growth rate. Seattle is the northernmost large city in the contiguous United States.

Rabat  is the capital city of Morocco and its third largest city with an urban population of approximately 580,000  and a metropolitan population of over 1.2 million. It is also the capital city of the Rabat-Salé-Kénitra administrative region.

Baltimore  is the largest city in the U.S. state of Maryland, and the 30th-most populous city in the United States.

Taguig  is a highly urbanized city located in the south-eastern portion of Metro Manila in the Philippines. From being a thriving fishing community along the shores of Laguna de Bay, it is now an important residential, commercial and industrial center. According to the 2015 census, Taguig is the seventh-most populous city in the Philippines, after Zamboanga City; it has a population of 804,915.

Memphis is a city located along the Mississippi River in the southwestern corner of the U.S. state of Tennessee. With an estimated 2017 population of 652,236, it is the cultural and economic center of West Tennessee and the greater Mid-South region that includes portions of neighboring Arkansas and Mississippi. Memphis is the seat of Shelby County, the most populous county in Tennessee. Approximately 315 square miles in area, Memphis is one of the most expansive cities in the United States and features a wide variety of landscapes and distinct neighborhoods.

Stuttgart  is the capital and largest city of the German state of Baden-Württemberg. Stuttgart is located on the Neckar river in a fertile valley known locally as the "Stuttgart Cauldron." It lies an hour from the Swabian Jura and the Black Forest. Its urban area has a population of 609,219, making it the sixth largest city in Germany. 2.7 million people live in the city's administrative region and another 5.3 million people in its metropolitan area, making it the fourth largest metropolitan area in Germany. The city and metropolitan area are consistently ranked among the top 20 European metropolitan areas by GDP; Mercer listed Stuttgart as 21st on its 2015 list of cities by quality of living, innovation agency 2thinknow ranked the city 24th globally out of 442 cities  and the Globalization and World Cities Research Network ranked the city as a Beta-status world city in their 2014 survey.

Edinburgh  is the capital city of Scotland and one of its 32 council areas. Historically part of the county of Midlothian, it is located in Lothian on the Firth of Forth's southern shore.

Milwaukee  is the largest city in the state of Wisconsin and the fifth-largest city in the Midwestern United States. The county seat of Milwaukee County, it is on Lake Michigan's western shore. Ranked by estimated 2014 population, Milwaukee was the 31st largest city in the United States. The city's estimated population in 2017 was 595,351. Milwaukee is the main cultural and economic center of the Milwaukee metropolitan area. It is also part of the larger Milwaukee-Racine-Waukesha combined statistical area, which had an estimated population of 2,026,243 in the 2010 census. Milwaukee is the second-most densely populated metropolitan area in the Midwest, surpassed only by Chicago.

Genoa  is the capital of the Italian region of Liguria and the sixth-largest city in Italy. In 2015, 594,733 people lived within the city's administrative limits. As of the 2011 Italian census, the Province of Genoa, which in 2015 became the Metropolitan City of Genoa, counted 855,834 resident persons. Over 1.5 million people live in the wider metropolitan area stretching along the Italian Riviera.

Nashville  is the capital and most populous city of the U.S. state of Tennessee and the seat of Davidson County. It is located on the Cumberland River in northern Middle Tennessee. The city is a center for the music, healthcare, publishing, private prison, banking and transportation industries, and is home to numerous colleges and universities.

Baltimore  is the largest city in the U.S. state of Maryland, and the 30th-most populous city in the United States.

Aden  is a port city in Yemen, located by the eastern approach to the Red Sea , some 170 km  east of Bab-el-Mandeb. Its population is approximately 800,000 people. Aden's natural harbour lies in the crater of a dormant volcano, which now forms a peninsula joined to the mainland by a low isthmus. This harbour, Front Bay, was first used by the ancient Kingdom of Awsan between the 5th and 7th centuries BC. The modern harbour is on the other side of the peninsula. Aden gives its name to the Gulf of Aden.

Bristol  is a city and county in South West England with a population of 456,000. The wider district has the 10th-largest population in England. The urban area population of 724,000 is the 8th-largest in the UK. The city borders North Somerset and South Gloucestershire, with the cities of Bath and Gloucester to the south-east and north-east, respectively. South Wales lies across the Severn estuary.

Dresden  is the capital city and, after Leipzig, the second-largest city of the Free State of Saxony in Germany. It is situated in a valley on the River Elbe, near the border with the Czech Republic.

Bradford   is in the Metropolitan Borough of the City of Bradford in West Yorkshire, England, in the foothills of the Pennines 8.6 miles  west of Leeds, and 16 miles  northwest of Wakefield. Bradford became a municipal borough in 1847, and received its charter as a city in 1897. Following local government reform in 1974, city status was bestowed upon the wider metropolitan borough.

Cardiff  is the capital of, and largest city in, Wales, and the eleventh-largest city in the United Kingdom. It is Wales's chief commercial centre, the base for most national cultural and sporting institutions, the Welsh national media, and the seat of the National Assembly for Wales. The unitary authority area's 2016 population was estimated to be 361,468, while the population of the Cardiff Capital Region  was estimated to be 1.52M. Cardiff is a significant tourist centre and the most popular visitor destination in Wales with 18.3 million visitors in 2010. In 2011 Cardiff was ranked sixth in the world in National Geographic's alternative tourist destinations.

Sheffield  is a city and metropolitan borough in South Yorkshire, England. Historically part of the West Riding of Yorkshire, its name derives from the River Sheaf, which runs through the city. With some of its southern suburbs annexed from Derbyshire, the city has grown from its largely industrial roots to encompass a wider economic base. The population of the City of Sheffield is 575,400  and it is one of the eight largest regional English cities that make up the Core Cities Group. Sheffield is the third largest English district by population. The metropolitan population of Sheffield is 1,569,000.

Islamabad  is the capital city of Pakistan located within the federal Islamabad Capital Territory. With a population of 2.01 million, it is the 9th largest city of Pakistan, while the larger Islamabad-Rawalpindi metropolitan area is the third largest in Pakistan with a population exceeding four million. The city is the political seat of Pakistan and is administered by the Islamabad Metropolitan Corporation, supported by the Capital Development Authority .

Denzil are a rock/powerpop band from Bournemouth, England, based around singer-songwriter Denzil Thomas. Denzil were signed to Giant Records in the USA in 1993 and put out one album, Pub, in 1994. Pub was nominated for two Grammy Awards in 1994.

Lyon , is the third-largest city and second-largest urban area of France. It is located in the country's east-central part at the confluence of the rivers Rhône and Saône, about 470 km  south from Paris, 320 km  north from Marseille and 55 km  east from Saint-Étienne. Inhabitants of the city are called Lyonnais.

Benghazi   is the second-most populous city in Libya and the largest in Cyrenaica.

Brno  is the second largest city in the Czech Republic by population and area, the largest Moravian city, and the historical capital city of the Margraviate of Moravia. Brno is the administrative center of the South Moravian Region in which it forms a separate district . The city lies at the confluence of the Svitava and Svratka rivers and has about 400,000 inhabitants; its greater metropolitan area is home to more than 800,000 people while its larger urban zone had a population of about 730,000 in 2004.

Bologna  is the capital and largest city of the Emilia-Romagna Region in Northern Italy. It is the seventh most populous city in Italy, at the heart of a metropolitan area of about one million people.

Florence  is the capital city of the Italian region of Tuscany. It is the most populous city in Tuscany, with 383,084 inhabitants in 2013, and over 1,520,000 in its metropolitan area.

Nice  is the fifth most populous city in France and the capital of the Alpes-Maritimes département. The urban area of Nice extends beyond the administrative city limits, with a population of about 1 million on an area of 721 km2 . Located in the French Riviera, on the south east coast of France on the Mediterranean Sea, at the foot of the Alps, Nice is the second-largest French city on the Mediterranean coast and the second-largest city in the Provence-Alpes-Côte d'Azur region after Marseille. Nice is about 13 kilometres  from the principality of Monaco, and its airport is a gateway to the principality as well.

Bilbao  is a city in northern Spain, the largest city in the province of Biscay and in the Basque Country as a whole. It is also the largest city proper in northern Spain. Bilbao is the tenth largest city in Spain, with a population of 345,141 as of 2015. The Bilbao metropolitan area has roughly 1 million inhabitants, making it one of the most populous metropolitan areas in northern Spain; with a population of 875,552 the comarca of Greater Bilbao is the fifth-largest urban area in Spain. Bilbao is also the main urban area in what is defined as the Greater Basque region.

Plymouth  is a city situated on the south coast of Devon, England, approximately 37 miles  south-west of Exeter and 190 miles  west-south-west of London. Enclosing the city are the mouths of the river Plym and river Tamar, which are naturally incorporated into Plymouth Sound to form a boundary with Cornwall.

Verona  is a city on the Adige river in Veneto, Italy, with approximately 257,000 inhabitants and one of the seven provincial capitals of the region. It is the second largest city municipality in the region and the third largest in northeast Italy. The metropolitan area of Verona covers an area of 1,426 km2  and has a population of 714,274 inhabitants. It is one of the main tourist destinations in northern Italy, owing to its artistic heritage, several annual fairs, shows, and operas, such as the lyrical season in the Arena, the ancient amphitheater built by the Romans.

Montpellier  is a city in southern France. It is the capital of the Hérault department. Montpellier is the 7th-largest city of France, and is also the fastest-growing city in the country over the past 25 years. In 2014, 589,610 people live in the urban area and 275,318 in the city itself. Nearly one third of the population are students from three universities and from three higher education institutions that are outside the university framework in the city. Located near the south coast of France on the Mediterranean Sea, it is the third-largest French city on the Mediterranean coast after Marseille and Nice.

Southampton  is the largest city in the ceremonial county of Hampshire, England. It is 69 miles  south-west of London and 15 miles  west north-west of Portsmouth Southampton is a major port and the closest city to the New Forest. It lies at the northernmost point of Southampton Water at the confluence of the Rivers Test and Itchen, with the River Hamble joining to the south of the urban area. The city, which is a unitary authority, has an estimated population of 253,651. The city's name is sometimes abbreviated in writing to "So'ton" or "Soton", and a resident of Southampton is called a Sotonian.

Wolverhampton  is a city and metropolitan borough in the West Midlands, England. At the 2011 census, it had a population of 249,470. The demonym for people from the city is "Wulfrunian".

Derby  is a city and unitary authority area in Derbyshire, England. It lies on the banks of the River Derwent in the south of Derbyshire, of which it was traditionally the county town. At the 2011 census, the population was 248,700. Derby gained city status in 1977.

Plymouth  is a city situated on the south coast of Devon, England, approximately 37 miles  south-west of Exeter and 190 miles  west-south-west of London. Enclosing the city are the mouths of the river Plym and river Tamar, which are naturally incorporated into Plymouth Sound to form a boundary with Cornwall.

Katowice  is a city in southern Poland, with a population of 297,197 as of 2017 and the center of the Silesian Metropolis, with a population of 2.2 million.

Alicante , or Alacant , both the Spanish and Valencian being official names, is a city and port in Spain on the Costa Blanca, the capital of the province of Alicante and of the comarca of Alacantí, in the south of the Valencian Community. It is also a historic Mediterranean port. The population of the city of Alicante proper was 330,525, estimated as of 2016, ranking as the second-largest Valencian city. Including nearby municipalities, the Alicante conurbation had 452,462 residents. The population of the metropolitan area  was 757,085 as of 2014 estimates, ranking as the eighth-largest metropolitan area of Spain.

Leicester  is a city and unitary authority area in the East Midlands of England, and the county town of Leicestershire. The city lies on the River Soar and close to the eastern end of the National Forest.

Wakefield is a city in West Yorkshire, England, on the River Calder and the eastern edge of the Pennines, which had a population of 99,251 at the 2011 census.

Bari   is the capital city of the Metropolitan City of Bari and of the Apulia region, on the Adriatic Sea, in southern Italy. It is the second most important economic centre of mainland Southern Italy after Naples, a port and university city, as well as the city of Saint Nicholas. The city itself has a population of 326,799, as of 2015, over 116 square kilometres , while the urban area has 700,000 inhabitants. The metropolitan area has 1.3 million inhabitants.

Kaunas  is the second-largest city in Lithuania and the historical centre of Lithuanian economic, academic, and cultural life. Kaunas was the biggest city and the centre of a county in Trakai Municipality of the Grand Duchy of Lithuania since 1413. In the Russian Empire, it was the capital of the Kaunas Governorate from 1843 to 1915. It served as temporary capital of Lithuania during the interwar period, while the traditional capital of Vilnius was part of Poland between 1920 and 1939. During that period Kaunas has been nicknamed the Little Paris because of rich cultural and academic life, fashion, construction of countless Art Deco, Lithuanian National Romanticism architectural style buildings as well as popular furniture, interior design of the time and widespread café culture. The city interwar architecture is regarded as one of the finest examples of the European Art Deco and received the European Heritage Label, it also resulted in naming Kaunas the first city in Central and Eastern Europe as a UNESCO City of Design. In 2022, Kaunas will be the European Capital of Culture, together with the Luxembourgish city of Esch-sur-Alzette.

The Federal City of Bonn  is a city on the banks of the Rhine in the German state of North Rhine-Westphalia, with a population of over 300,000. About 24 km  south-southeast of Cologne, Bonn is in the southernmost part of the Rhine-Ruhr region, Germany's largest metropolitan area, with over 11 million inhabitants.

Valladolid  is a city in Spain and the de facto capital of the autonomous community of Castile and León. It has a population of 309,714 people , making it Spain's 13th most populous municipality and northwestern Spain's biggest city. Its metropolitan area ranks 20th in Spain with a population of 414,244 people in 23 municipalities.

Coventry  is a city and metropolitan borough in the West Midlands, England.

Nottingham  is a city and unitary authority area in Nottinghamshire, England, 128 miles  north of London, in the East Midlands.

Amman  is the capital and most populous city of Jordan, and the country's economic, political and cultural centre. Situated in north-central Jordan, Amman is the administrative centre of the Amman Governorate. The city has a population of 4,007,526 and a land area of 1,680 square kilometres . Today, Amman is considered to be among the most liberal and westernized Arab cities. It is a major tourist destination in the region, particularly among Arab and European tourists.

Heze, formerly known as Caozhou, is a prefecture-level city in southwestern Shandong, China. It is home to 8,287,693 inhabitants, of whom 1,346,717 live in the built-up area around the seat of government in Mudan District. The westernmost prefecture-level city in Shandong, it borders Jining to the east and the provinces of Henan and Anhui to the west and south respectively.

Ningbo , formerly written Ningpo, is a sub-provincial city in northeast Zhejiang province in China. It comprises the urban districts of Ningbo proper, three satellite cities, and a number of rural counties including islands in Hangzhou Bay and the East China Sea. Its port, spread across several locations, is among the busiest in the world and the municipality possesses a separate state-planning status. As of the 2010 census, the entire administrated area had a population of 7.6 million, with 3.5 million in the six urban districts of Ningbo proper. To the north, Hangzhou Bay separates Ningbo from Shanghai; to the east lies Zhoushan in the East China Sea; on the west and south, Ningbo borders Shaoxing and Taizhou respectively.

Almaty  , formerly known as Alma-Ata   and Verny , is the largest city in Kazakhstan, with a population of 1,797,431 people, about 8% of the country's total population. It served as capital of the Kazakh state in its various forms from 1929 to 1997, under the influence of the then Soviet Union and its appointees. Alma-Ata was the host city for a 1978 international conference on Primary Health Care where the Alma Ata Declaration was adopted, marking a paradigm shift in global public health. In 1997, the government relocated the capital to Astana in the north of the country, which is about 12 hours away by train.

Hefei  is the capital and largest city of Anhui Province in China. A prefecture-level city, it is the political, economic, and cultural center of Anhui. Located in the central portion of the province, it borders Huainan to the north, Chuzhou to the northeast, Wuhu to the southeast, Tongling to the south, Anqing to the southwest and Lu'an to the west.

Laiwu  is a prefecture-level city in central Shandong Province, China. The smallest prefecture-level city in the province, it borders the provincial capital of Jinan to the north, Zibo to the east and Tai'an to the southwest. Its population was 1,298,529 at the 2010 census whom 989,535 lived in the built-up area made up of Laicheng district, Gangcheng district not being conurbated yet.

Jiangyin  is a county-level city on the southern bank of the Yangtze River, and is administered by Wuxi, Jiangsu province. Jiangyin is one of the most important transport hubs on the Yangtze River, it is also one of the most developed counties in China.

Indore   is the most populous and the largest city in the Indian state of Madhya Pradesh. It serves as the headquarters of both Indore District and Indore Division. It is also considered as an education hub of the state and first city to have campuses of both the Indian Institute of Technology and the Indian Institute of Management. Located on the southern edge of Malwa Plateau, at an average altitude of 550 meters above sea level it has the highest elevation among major cities of Central India, the city is 190 km west of the state capital of Bhopal. With a census-estimated 2011 population of 1,994,397  and 2,170,295 , the Indore Metropolitan Area's population is the state's largest. The city is distributed over a land area of just 530 square kilometers making Indore the most densely populated major city in the central province. It comes under Tier 2 cities in India. It has been ranked first in Swachh Bharat Abhiyan two years in a row and is the cleanest city in India.

Kazan  is the capital and largest city of the Republic of Tatarstan, Russia. With a population of 1,143,535, it is the sixth most populous city in Russia. Kazan lies at the confluence of the Volga and Kazanka Rivers in European Russia. The Kazan Kremlin is a World Heritage Site.

Yixing  is a county-level city with a population of 1.24 million administrated under the prefecture-level city of Wuxi in southern Jiangsu province, People's Republic of China, and is part of the Yangtze River Delta. The city is known for its traditional Yixing clay ware.

Leshan, formerly known as Jiading, is a prefecture-level city located at the confluence of the Dadu and Min rivers in Sichuan Province, China. Leshan is located on the southwestern fringe of the Red Basin in southern Sichuan, about 120 km  from Chengdu. At the 2010 census, its population was 3,235,759 whom 662,814 lived in the built-up  area made of Shizhong district, as Wutongqiao and Jinkouhe districts are not conurbated yet.

Shiraz  is the fifth-most-populous city of Iran and the capital of Fars Province . At the 2011 census, the population of the city was 1,700,665 and its built-up area with "Shahr-e Jadid-e Sadra"  was home to 1,500,644 inhabitants. Shiraz is located in the southwest of Iran on the "Roodkhaneye Khoshk"  seasonal river. It has a moderate climate and has been a regional trade center for over a thousand years. Shiraz is one of the oldest cities of ancient Persia.

Rui'an  is a county-level city along the southern coast of Zhejiang province, China, and is under the administration of Wenzhou City. It has a population of 1,125,000 people and covers a land area of 1,271 km2 , 3,037 km2  when including water area. Natives of the city speak the Rui'an dialect of Wu Chinese.

Jimbo is a diminutive form of the given name James. It is also a Japanese surname. It may refer to:

Volgograd , formerly Tsaritsyn , 1589–1925, and Stalingrad , 1925–1961, is an important industrial city and the administrative centre of Volgograd Oblast, Russia, on the western bank of the Volga River. The Battle of Stalingrad in World War II was the largest and bloodiest battle in the history of warfare.

Chifeng , also known as Ulankhad , is a prefecture-level city in southeastern Inner Mongolia, People's Republic of China. It borders Xilin Gol League to the north and west, Tongliao to the northeast, Chaoyang  to the southeast, and Chengde  to the south. The city has a total administrative area of 90,275 square kilometres  and has a population of 4,341,245 inhabitants. As of the 2010 census, 1,094,970 of those residents reside within in the urban districts of Hongshan, Yuanbaoshan and Songshan. However, a large part of Songshan is still rural and Yuanbaoshan is a de facto separate town 27 kilometers away from the core district of Chifeng. The city was the administrative center of the defunct Ju Ud League .

Qinzhou  is a prefecture-level city in Guangxi, China, lying on the Gulf of Tonkin and having an urban population of 433,000.

Odessa  is the third most populous city of Ukraine and a major tourism center, seaport and transportation hub located on the northwestern shore of the Black Sea. It is also the administrative center of the Odessa Oblast and a multiethnic cultural center. Odessa is sometimes called the "pearl of the Black Sea", the "South Capital" , and "Southern Palmyra".

Pekalongan  is a city of Central Java, Indonesia. It was formerly the seat of Pekalongan Regency on the northern coast of the province, but is now an independent municipality within the province. The city is Central Java's most important port, and is known for its batik. Since December 2014, Pekalongan is a member of UNESCO’s World’s Creative Cities Network. Pekalongan is the first Indonesian city and first South East Asian city listed as member of UNESCO’s World’s Creative Cities Network

Almaty  , formerly known as Alma-Ata   and Verny , is the largest city in Kazakhstan, with a population of 1,797,431 people, about 8% of the country's total population. It served as capital of the Kazakh state in its various forms from 1929 to 1997, under the influence of the then Soviet Union and its appointees. Alma-Ata was the host city for a 1978 international conference on Primary Health Care where the Alma Ata Declaration was adopted, marking a paradigm shift in global public health. In 1997, the government relocated the capital to Astana in the north of the country, which is about 12 hours away by train.

Guangzhou , also known as Canton, is the capital and most populous city of the province of Guangdong. Located in southern China on the Pearl River about 120 km  north-northwest of Hong Kong and 145 km  north of Macau, Guangzhou has a history of over 2,200 years and was a major terminus of the maritime Silk Road and continues to serve as a major port and transportation hub today, as well as one of China's three largest cities.

Tehran  is the capital of Iran and Tehran Province. With a population of around 8.8 million in the city and 15 million in the larger metropolitan area of Greater Tehran, Tehran is the most populous city in Iran and Western Asia, and has the second-largest metropolitan area in the Middle East. It is ranked 29th in the world by the population of its metropolitan area.

Monterrey  is the capital and largest city of the northeastern state of Nuevo León, Mexico. The city is anchor to the third-largest metropolitan area in Mexico and is the country's ninth-largest city. Monterrey serves as a commercial center of northern Mexico and is the base of many significant international corporations.

Cape Town  is a coastal city in South Africa. It is the second-most populous urban area in South Africa after Johannesburg. It is the capital and primate city of the Western Cape province.

Birmingham , locally sometimes: ) is a city and metropolitan borough in the West Midlands, England, with an estimated population of 1,101,360 as of 2014, making it the most populous city proper in the country. It is the main centre of the West Midlands conurbation, which is the third most populated urban area in the United Kingdom, with a population in 2011 of 2,440,986,. The wider Birmingham Metropolitan Area is the second largest in the United Kingdom, with a population of 3.7 million. Birmingham is frequently referred to as the second city of England.

Wuhan  is the capital of Hubei province, People's Republic of China. It's the most populous city in Central China, and one of the nine National Central Cities of China. It lies in the eastern Jianghan Plain on the middle reaches of the Yangtze River's intersection with the Han river. Arising out of the conglomeration of three cities, Wuchang, Hankou, and Hanyang, Wuhan is known as 'China's Thoroughfare'; it is a major transportation hub, with dozens of railways, roads and expressways passing through the city and connecting to other major cities. Because of its key role in domestic transportation, Wuhan is sometimes referred to as "the Chicago of China" by foreign sources.

Tianjin , formerly romanized as Tientsin, is a coastal metropolis in northern China and one of the four national central cities of the People's Republic of China , with a total population of 15,469,500, and is also the world's 11th-most populous city proper. It is governed as one of the four direct-controlled municipalities of the PRC and is thus under direct administration of the central government. Tianjin borders Hebei Province and Beijing Municipality, bounded to the east by the Bohai Gulf portion of the Yellow Sea. Part of the Bohai Economic Rim, it is the largest coastal city in northern China.

"""

In [0]:
#Converting the Text to lower case
raw_text=text.lower()

In [0]:
# mapping each character
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [0]:
# Total vocabulory 70
# TOtal characters 111109
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

In [0]:
#the lengeth of particular sequence which can be changed at will
#and finding patterns if possible among the sequences

seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)

In [0]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [0]:
# defining the model here
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [0]:
#checkpoint and reduce lr 
from keras.callbacks import ReduceLROnPlateau
filepath="text3.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5,
              patience=1, min_lr=0.00001)
callbacks_list = [checkpoint,reduce_lr]

In [0]:
# Experimented with diff values optimizer
#optimizer = keras.optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer='Adam')

In [0]:
#training done here for 2 layer around 25 epochs
epoch=0
epochs=25

model.fit(X, y, epochs=epochs,initial_epoch=epoch, batch_size=64, callbacks=callbacks_list)
model.save_weights("text2.h5")
print("Saved model to disk")

In [0]:
# Generating seed value to start with
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

In [0]:
#Generate random value

for i in range(300):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")